## Import libraries

In [1]:
# Importing required libraries
from fastcore.all import AttrDict
import io
from pdf2image import convert_from_path
from pypdf import PdfReader
import openai
import os
import base64
import requests
from dotenv import load_dotenv
import pandas as pd 
import json
import re
import zipfile
from pathlib import Path 
import unicodedata

load_dotenv()  # Carga las variables de entorno desde .env

True

## Load moodle student information
You must have downloaded
- All deriverables
- The full list of studens

In [10]:
# Load moodle students information
import pandas as pd

# Cargar el CSV de alumnos y grupos
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [ pd.read_csv(filename) for filename in students_info ]

df_students = pd.concat(dfs, ignore_index=True)

# Puedes opcionalmente limpiar espacios y convertir a mayúsculas para facilitar coincidencias
df_students["Nombre"] = df_students["Nombre"].str.strip().str.upper()
df_students["Apellido(s)"] = df_students["Apellido(s)"].str.strip().str.upper()

json_students = "\n".join(
    f"{row['Nombre']} {row['Apellido(s)']} - Grupo: {row['Grupos']}"
    for _, row in df_students.iterrows()
)

In [17]:
api_key = os.environ["OPENAI_API_KEY"] # *
prompt = """Extract the last name (Apellidos in Spanish), the first name (Nombre in Spanish) and the group (Grupo in Spanish)
 from the top of the image. You will find them handwritten after the labels `Apellidos`,  `Nombre` and `Grupo` respectively. The fields of your JSON output will have those exact same label names
 Here is a list of expected students and their groups as a reference: 
 {json_info}
 """ # *
model = "gpt-4o"

In [4]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [5]:
def pdf_to_base64(pdf_path, crop_height=450):
    images = convert_from_path(pdf_path)
    images = [
        image.crop((0, 0, image.width, crop_height))
        for image in images
    ]
    return images

In [6]:
def img_openai_payload(image, model, prompt):
    payload = {
    "model": f'{model}',
    "response_format": { "type": "json_object" },
    "messages": [
        {
            "role": "system", 
            "content": "You are a helpful assistant designed to see an exam and output JSON \
            with the extracted information. You will be given an image of the exam.\
            the default group in case of empty string is extraviado",   
        },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": f"{prompt}"
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image}"
                }
                }
            ]
            }
    ],
    "max_tokens": 300
    }
    return payload

In [7]:
def pdf_openai_parser(pdf_path, model, prompt, json_info = None):
    images = pdf_to_base64(pdf_path, crop_height=450)
    
    for i, image in enumerate(images):
        # Convert image to base64
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        
        # Create payload for this image
        payload = img_openai_payload(img_str, model, prompt.format(guia_texto=json_info))
        
        # Make API request
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        
        if response.status_code == 200:
            result = response.json()
            try:
                # Parse the JSON response
                extracted_data = json.loads(result['choices'][0]['message']['content'])
                
                # Check if we have valid data (Apellidos, Nombre, and Grupo)
                if all(key in extracted_data and extracted_data[key].strip() for key in ['Apellidos', 'Nombre', 'Grupo']):
                    print(f"Successfully extracted data from image {i+1}")
                    return extracted_data
                else:
                    print(f"Image {i+1} missing required fields, trying next image...")
                    
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error parsing response from image {i+1}: {e}, trying next image...")
        else:
            print(f"API request failed for image {i+1}: {response.status_code}")
    
    print("No valid data found in any image")
    return None

# Parse 1 exam

In [8]:
def pdf_openai_parser_safeGroup(pdf_path, model, prompt, json_info = None):
    student_info = pdf_openai_parser(
        pdf_path, 
        model = model, 
        prompt = prompt,
        json_info = json_info
    )
    if not student_info.get("Grupo"):
        student_info["Grupo"] = "extraviado"
    return student_info

In [18]:
pdf_openai_parser_safeGroup(
    pdf_path    = "../example_data/1.pdf",
    model       = model,
    prompt      = prompt.format(json_info=json_students)
)

Successfully extracted data from image 1


{'Apellidos': 'RODRIGUEZ FERNANDEZ', 'Nombre': 'VICTOR', 'Grupo': 'Profesores'}

# Process Scan

You must have the scanneed pdf into the '../data/raw/ folder'

In [19]:
# Para debug más fácil
def renombrar_archivos_en_lotes(ruta="../data/raw/"):
	"""
	Renombra todos los archivos en la carpeta dada como 'lote_1', 'lote_2', etc.
	Conserva la extensión original de cada archivo.
	"""
	archivos = sorted([f for f in os.listdir(ruta) if os.path.isfile(os.path.join(ruta, f))])
	for idx, nombre_original in enumerate(archivos, start=1):
		extension = os.path.splitext(nombre_original)[1]
		nuevo_nombre = f"lote_{idx}{extension}"
		ruta_origen = os.path.join(ruta, nombre_original)
		ruta_destino = os.path.join(ruta, nuevo_nombre)
		os.rename(ruta_origen, ruta_destino)
	print(f"Renombrados {len(archivos)} archivos en '{ruta}'.")

In [20]:
#renombrar_archivos_en_lotes()

In [21]:
# Si estás seguro de tu scanner usa esta función, lo lo recomiendo
import os
from pathlib import Path
from pypdf import PdfReader, PdfWriter

def crear_carpeta_examenes(base_dir="../data", nombre_base="examenes"):
    """
    Crea una carpeta nueva para los exámenes. Si ya existe, añade un sufijo numérico.
    """
    base_path = Path(base_dir)
    carpeta = base_path / nombre_base
    contador = 1
    while carpeta.exists():
        carpeta = base_path / f"{nombre_base}_{contador}"
        contador += 1
    carpeta.mkdir(parents=True)
    return carpeta

def dividir_pdf_en_examenes(pdf_path, carpeta_destino, nombre_base="examen"):
    """
    Divide un PDF en archivos de 2 páginas cada uno y los guarda en la carpeta destino.
    """
    reader = PdfReader(pdf_path)
    num_paginas = len(reader.pages)
    examen_idx = 1
    for i in range(0, num_paginas, 2):
        writer = PdfWriter()
        writer.add_page(reader.pages[i])
        if i+1 < num_paginas:
            writer.add_page(reader.pages[i+1])
        nombre_examen = f"{nombre_base}_{examen_idx}.pdf"
        ruta_examen = carpeta_destino / nombre_examen
        with open(ruta_examen, "wb") as f_out:
            writer.write(f_out)
        examen_idx += 1

def procesar_lotes_y_generar_examenes(ruta_lotes="../data/raw/", base_dir="../data", nombre_carpeta="examenes"):
    """
    Busca todos los archivos PDF en la carpeta de lotes, los divide de 2 en 2 páginas y los guarda en una carpeta nueva.
    """
    carpeta_destino = crear_carpeta_examenes(base_dir, nombre_carpeta)
    archivos_lote = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    examen_global_idx = 1
    for archivo in archivos_lote:
        ruta_pdf = Path(ruta_lotes) / archivo
        reader = PdfReader(ruta_pdf)
        num_paginas = len(reader.pages)
        for i in range(0, num_paginas, 2):
            writer = PdfWriter()
            writer.add_page(reader.pages[i])
            if i+1 < num_paginas:
                writer.add_page(reader.pages[i+1])
            nombre_examen = f"examen_{examen_global_idx}.pdf"
            ruta_examen = carpeta_destino / nombre_examen
            with open(ruta_examen, "wb") as f_out:
                writer.write(f_out)
            examen_global_idx += 1
    print(f"Exámenes generados en: {carpeta_destino}")

# Ejemplo de uso:
#procesar_lotes_y_generar_examenes()

In [22]:
import os
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader, PdfWriter
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown
from IPython.display import display, clear_output

class JupyterPDFReviewer:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.reader = PdfReader(pdf_path)
        self.total_pages = len(self.reader.pages)
        self.current_index = 0
        self.images = {}
        self.output_dir = Path("../data/saved")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.out = Output()
        self.status = Label(value="")  # Estado visual
        self._setup_widgets()
        self._show_pages()

    def _setup_widgets(self):
        self.btn_save1 = Button(description='Save Page 1', layout=Layout(width='120px'))
        self.btn_save12 = Button(description='Save Pages 1&2', layout=Layout(width='120px'))
        self.btn_save123 = Button(description='Save Pages 1-3', layout=Layout(width='120px'))
        self.btn_next = Button(description='Next (Skip 1)', layout=Layout(width='120px'))
        self.btn_prev = Button(description='Previous', layout=Layout(width='120px'))

        self.btn_save1.on_click(lambda x: self._save_pages([0]))
        self.btn_save12.on_click(lambda x: self._save_pages([0, 1]))
        self.btn_save123.on_click(lambda x: self._save_pages([0, 1, 2]))
        self.btn_next.on_click(lambda x: self._next_page())
        self.btn_prev.on_click(lambda x: self._prev_page())

        display(VBox([
            HBox([self.btn_prev, self.btn_save1, self.btn_save12, self.btn_save123, self.btn_next]),
            self.status,
            self.out
        ]))

    def _get_page_image(self, idx):
        if idx not in self.images and idx < self.total_pages:
            self.status.value = f"Cargando página {idx+1}..."
            try:
                img = convert_from_path(
                    self.pdf_path,
                    first_page=idx + 1,
                    last_page=idx + 1,
                    dpi=50,
                    fmt='jpeg',
                    thread_count=1
                )[0]
                self.images[idx] = img
            except Exception as e:
                self.status.value = f"Error cargando página {idx+1}"
                print(f"Error converting page {idx+1}: {e}")
                return None
        self.status.value = ""
        return self.images.get(idx)

    def _show_pages(self):
        with self.out:
            clear_output(wait=True)
            fig, axes = plt.subplots(1, 3, figsize=(15, 8))
            for i in range(3):
                page_idx = self.current_index + i
                axes[i].axis('off')
                if page_idx < self.total_pages:
                    img = self._get_page_image(page_idx)
                    if img is not None:
                        axes[i].imshow(img)
                        axes[i].set_title(f"Page {page_idx+1}")
                    else:
                        axes[i].set_title(f"Page {page_idx+1} (error)")
                else:
                    axes[i].set_title("No Page")
            plt.show()

    def _next_page(self):
        if self.current_index + 1 < self.total_pages:
            self.current_index += 1
            self._show_pages()

    def _prev_page(self):
        if self.current_index >= 1:
            self.current_index -= 1
            self._show_pages()

    def _save_pages(self, rel_indices):
        abs_indices = [self.current_index + i for i in rel_indices if self.current_index + i < self.total_pages]
        if not abs_indices:
            self.status.value = "No valid pages to save"
            return

        base_name = Path(self.pdf_path).stem  # Ejemplo: 'lote_1'
        lote = base_name
        examen_n = abs_indices[0] + 1  # Primer índice de página + 1
        output_path = self.output_dir / f"{lote}_examen_{examen_n}.pdf"

        writer = PdfWriter()
        for idx in abs_indices:
            writer.add_page(self.reader.pages[idx])
        with open(output_path, "wb") as f:
            writer.write(f)
        self.status.value = f"Guardado: {output_path.name}"

        # Avanzar tantas páginas como se han guardado
        avance = len(abs_indices)
        if self.current_index + avance < self.total_pages:
            self.current_index += avance
            self._show_pages()

def revisar_todos_los_lotes(ruta_lotes="../data/raw/"):
    archivos = sorted([f for f in os.listdir(ruta_lotes) if f.lower().endswith(".pdf")])
    if not archivos:
        print("No se encontraron lotes PDF en la carpeta.")
        return
    dropdown = Dropdown(options=archivos, description='Lote:', layout=Layout(width='50%'))
    out = Output()

    def on_select(change):
        with out:
            clear_output(wait=True)
            print(f"Revisando: {dropdown.value}")
            JupyterPDFReviewer(os.path.join(ruta_lotes, dropdown.value))

    dropdown.observe(on_select, names='value')
    display(VBox([dropdown, out]))
    # Mostrar el primero por defecto
    on_select({'new': archivos[0]})


In [23]:
#revisar_todos_los_lotes("../data/raw/") # --> solo cuando necesites procesar los lotes

## Procesar examenes con OpenAI

In [24]:
import os
import json
import base64
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
import pandas as pd
import shutil
import io
import requests
from rapidfuzz import fuzz
import re
import unicodedata


In [25]:
def limpiar_texto(texto):
    """Elimina acentos, símbolos y deja solo letras/números/espacios en mayúsculas"""
    if not texto:
        return ""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')
    texto = texto.upper()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    texto = texto.strip()
    return texto

In [26]:
def buscar_grupo_flexible(nombre, apellidos, df, texto_ocr=None, umbral=90):
    """
    Busca el grupo del alumno en el DataFrame por nombre y apellidos usando fuzzy.
    Si no encuentra por el umbral, devuelve el grupo más parecido aunque el score sea bajo (mínimo 20%).
    """
    nombre = limpiar_texto(nombre)
    apellidos = limpiar_texto(apellidos)
    mejor_score = -1
    mejor_grupo = None
    mejor_nombre = ""
    mejor_apellidos = ""
    for _, row in df.iterrows():
        nombre_df = limpiar_texto(str(row["Nombre"]))
        apellidos_df = limpiar_texto(str(row["Apellido(s)"]))
        score_nombre = fuzz.ratio(nombre, nombre_df)
        score_apellidos = fuzz.ratio(apellidos, apellidos_df)
        score = (score_nombre + score_apellidos) / 2
        if score > mejor_score:
            mejor_score = score
            mejor_grupo = row["Grupos"]
            mejor_nombre = row["Nombre"]
            mejor_apellidos = row["Apellido(s)"]
    # Si supera el umbral, devuelve el grupo y nombre exactos
    if mejor_score >= umbral:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no supera el umbral pero hay algún match > 20%, devuelve el más parecido
    if mejor_score >= 20:
        return mejor_grupo, mejor_nombre, mejor_apellidos
    # Si no hay nada ni con 20%, busca patrón OCR
    if texto_ocr:
        texto_ocr = texto_ocr.upper()
        patrones = [
            r"CITI[TM][1][12]", r"IWSI[TM][1][12]", r"IWSIT[1][12]", r"CITIT[1][12]", r"IWSIM[1][12]"
        ]
        for patron in patrones:
            match = re.search(patron, texto_ocr)
            if match:
                return match.group(0), "", ""
    return "extraviado", "", ""


In [27]:
def extraer_info_con_openai(base64_image, headers, df):
    try:
        prompt_completo = (
            "Extract the last name (Apellidos in Spanish), "
            "the first name (Nombre in Spanish) from the top of the image. "
            "Extract also the group (Grupo in Spanish) from the top of the image. "
            "You will find them handwritten after the labels `Apellidos` and `Nombre` and `Group` respectively. "
            "The fields of your JSON output will have those exact same label names. "
            "Also determine if this is a \"Práctica de Listas\" (practice 3) or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Add a field called \"practica_detectada\" with value 3 for listas or 5 for grafos. "
            "If you see a group label like CITIM11, CITIM12, IWSIM11, IWSIM12, CITIT11, IWSIT11, IWSIT12, include it as the field 'Grupo'."
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant designed to see an exam and output JSON with the extracted information."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_completo},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 300
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI: {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                nombre_ocr = info.get("Nombre", "")
                apellidos_ocr = info.get("Apellidos", "")
                grupo_ocr = info.get("Grupo", "")
                grupo, nombre, apellidos = buscar_grupo_flexible(nombre_ocr, apellidos_ocr, df, texto_ocr=grupo_ocr)
                info["Grupo"] = grupo
                # Si se encontró en el excel, usa los nombres del excel (más limpios)
                if grupo != "extraviado" and nombre and apellidos:
                    info["Nombre"] = nombre
                    info["Apellidos"] = apellidos
                else:
                    # Si no, limpia los nombres extraídos por OCR
                    info["Nombre"] = limpiar_texto(nombre_ocr)
                    info["Apellidos"] = limpiar_texto(apellidos_ocr)
                if grupo == "extraviado" and grupo_ocr:
                    info["Grupo_detectado"] = grupo_ocr
                return info, None
            except json.JSONDecodeError as e:
                return None, f"Error parseando JSON: {e}\nRespuesta recibida: {info_str}"
        else:
            return None, f"Error en API OpenAI: {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error extrayendo información: {e}"



In [28]:
def extraer_solo_practica(base64_image, headers):
    try:
        prompt_practica = (
            "Look at this exam image and determine if this is a \"Práctica de Listas\" (practice 3) "
            "or \"Práctica de Grafos\" (practice 5) based on the content. "
            "Return JSON with fields: \"Apellidos\": \"\", \"Nombre\": \"\", \"Grupo\": \"extraviado\", \"practica_detectada\": 3 or 5"
        )
        payload = {
            "model": "gpt-4o",
            "response_format": {"type": "json_object"},
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant that identifies exam types and outputs JSON.",
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_practica},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    ]
                }
            ],
            "max_tokens": 150
        }
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=10
            )
        except requests.exceptions.Timeout:
            return None, "Timeout: La petición a OpenAI tardó más de 10 segundos (solo práctica)."
        except requests.exceptions.RequestException as e:
            return None, f"Error de conexión con OpenAI (solo práctica): {e}"

        if response.status_code == 200:
            info_str = response.json()['choices'][0]['message']['content']
            try:
                info = json.loads(info_str)
                return info, None
            except json.JSONDecodeError:
                return None, f"Error parseando JSON (solo práctica): {info_str}"
        else:
            return None, f"Error en API OpenAI (solo práctica): {response.status_code} - {response.text}"
    except Exception as e:
        return None, f"Error en retry de práctica: {e}"


In [29]:
def extraer_numero_lote(nombre):
    match = re.search(r"lote_(\d+)", nombre)
    return int(match.group(1)) if match else 0

In [30]:
def mover_archivo_organizado(archivo_original, info, output_path):
    try:
        practica = info.get('practica_detectada', 'desconocida')
        grupo = info.get('Grupo', 'extraviado')
        carpeta_grupo = output_path / grupo
        carpeta_practica = carpeta_grupo / f"Practica_{practica}"
        carpeta_practica.mkdir(parents=True, exist_ok=True)
        apellidos = info.get('Apellidos', '').strip()
        nombre = info.get('Nombre', '').strip()
        if grupo == "extraviado" and not apellidos and not nombre:
            nombre_base = Path(info.get('archivo_original', archivo_original.name)).stem
        else:
            apellidos = apellidos if apellidos else 'SinApellidos'
            nombre = nombre if nombre else 'SinNombre'
            nombre_base = f"{apellidos.replace(' ', '_')}_{nombre.replace(' ', '_')}"
            if grupo == "extraviado" and info.get('Grupo_detectado'):
                nombre_base += f"_{info['Grupo_detectado']}"
        extension = archivo_original.suffix
        nuevo_archivo = carpeta_practica / f"{nombre_base}{extension}"
        contador = 2
        while nuevo_archivo.exists():
            nuevo_archivo = carpeta_practica / f"{nombre_base}_{contador}{extension}"
            contador += 1
        shutil.copy2(archivo_original, nuevo_archivo)
        print(f"  → Guardado en: {carpeta_grupo.name}/{carpeta_practica.name}/{nuevo_archivo.name}")
    except Exception as e:
        print(f"Error organizando archivo {archivo_original.name}: {e}")


In [31]:
def crear_dataframe_examenes(examenes_info, df):
    df_base = df[['Nombre', 'Apellido(s)', 'Grupos']].copy()
    df_base['Examen_3'] = 0
    df_base['Comentario_Examen_3'] = 'PNP'
    df_base['Examen_5'] = 0
    df_base['Comentario_Examen_5'] = 'PNP'
    for examen in examenes_info:
        practica = examen.get('practica_detectada')
        if practica in [3, 5, '3', '5']:
            practica = str(practica)
            apellidos = examen.get('Apellidos', '').upper().strip()
            nombre = examen.get('Nombre', '').upper().strip()
            if apellidos and nombre:
                mask = df_base['Apellido(s)'].str.upper().str.strip() == apellidos
                mask &= df_base['Nombre'].str.upper().str.strip() == nombre
                if mask.any():
                    df_base.loc[mask, f'Examen_{practica}'] = 1
                    df_base.loc[mask, f'Comentario_Examen_{practica}'] = ''
                    print(f"  → Marcado como presentado: {nombre} {apellidos} - Práctica {practica}")
    return df_base


In [32]:

def procesar_examenes_completo(carpeta_examenes="../data/saved/", output_dir="../data/examenes_procesados/"):
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    extraviados_path = output_path / "extraviados"
    extraviados_path.mkdir(parents=True, exist_ok=True)
    examenes_info = []
    errores_openai = []
    # Ordenar por número de lote, no alfabéticamente
    pdf_files = sorted(Path(carpeta_examenes).glob("*.pdf"), key=lambda x: extraer_numero_lote(x.name))
    print(f"Procesando {len(pdf_files)} exámenes...")
    api_key = os.environ["OPENAI_API_KEY"]
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    # Cargar DataFrame de alumnos
    students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
    dfs = [ pd.read_csv(filename) for filename in students_info ]
    df = pd.concat(dfs, ignore_index=True)
    df["Nombre"] = df["Nombre"].str.strip().str.upper()
    df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
    for pdf_file in pdf_files:
        print(f"\nProcesando: {pdf_file.name}")
        try:
            reader = PdfReader(pdf_file)
            num_pages = len(reader.pages)
            info_extraida = None
            error_detalle = None
            for page_num in range(min(2, num_pages)):
                images = convert_from_path(
                    pdf_file,
                    first_page=page_num + 1,
                    last_page=page_num + 1,
                    dpi=150,
                    fmt='jpeg'
                )
                if not images:
                    continue
                buffered = io.BytesIO()
                images[0].save(buffered, format="JPEG")
                base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
                info, error = extraer_info_con_openai(base64_image, headers, df)
                if info and info.get('Apellidos') and info.get('Nombre'):
                    info['archivo_original'] = pdf_file.name
                    info['pagina'] = page_num + 1
                    info_extraida = info
                    break
                elif info and not info.get('Apellidos'):
                    info_retry, error_retry = extraer_solo_practica(base64_image, headers)
                    if info_retry:
                        info_extraida = info_retry
                        info_extraida['archivo_original'] = pdf_file.name
                        info_extraida['pagina'] = page_num + 1
                        break
                    elif error_retry:
                        error_detalle = error_retry
                elif error:
                    error_detalle = error
            if info_extraida:
                mover_archivo_organizado(pdf_file, info_extraida, output_path)
                examenes_info.append(info_extraida)
                practica = info_extraida.get('practica_detectada', 'desconocida')
                print(f"✓ Extraído: {info_extraida.get('Nombre', 'Sin nombre')} {info_extraida.get('Apellidos', 'Sin apellidos')} - Grupo: {info_extraida.get('Grupo', 'extraviado')} - Práctica: {practica}")
            else:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"✗ No se pudo extraer información - Copiado a extraviados: {pdf_file.name}")
                errores_openai.append({
                    "archivo": pdf_file.name,
                    "error": error_detalle or "No se pudo extraer información ni con retry",
                })
        except Exception as e:
            print(f"✗ Error procesando {pdf_file.name}: {e}")
            try:
                shutil.copy2(pdf_file, extraviados_path / pdf_file.name)
                print(f"  → Copiado a extraviados por error")
            except:
                pass
            errores_openai.append({
                "archivo": pdf_file.name,
                "error": str(e),
            })
            continue
    df_examenes = crear_dataframe_examenes(examenes_info, df)
    df_examenes.to_csv(output_path / "seguimiento_examenes.csv", index=False)
    print(f"\n📊 Procesamiento completado:")
    print(f"- Exámenes procesados: {len(examenes_info)}")
    print(f"- Archivos en extraviados: {len(list(extraviados_path.glob('*.pdf')))}")
    print(f"- Archivo de seguimiento guardado en: {output_path / 'seguimiento_examenes.csv'}")
    # Crear DataFrame de errores y mostrarlo
    if errores_openai:
        df_errores = pd.DataFrame(errores_openai)
        df_errores.to_csv(output_path / "errores_openai.csv", index=False)
        print(f"\n❗ Casos fallidos por OpenAI guardados en: {output_path / 'errores_openai.csv'}")
        print(df_errores)
    else:
        print("\n✅ No hubo errores de extracción con OpenAI.")
    return df_examenes

In [33]:
# Ejecutar el procesamiento
#df_examenes_procesados = procesar_examenes_completo()
print("\n📊 Resumen de exámenes procesados por grupo:")
#print(df_examenes_procesados.groupby('Grupos')[['Examen_3', 'Examen_5']].sum())


📊 Resumen de exámenes procesados por grupo:


### Versión visual (más segura con diferencia)

In [34]:
import os
import shutil
from pathlib import Path
from pdf2image import convert_from_path
from pypdf import PdfReader
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Combobox, Dropdown, HTML, ToggleButtons
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from difflib import get_close_matches
import re

# --- Cargar el DataFrame de alumnos (ajusta la ruta si es necesario) ---
students_info = ["./../data/courseid_422_participants.csv", "./../data/courseid_23101_participants.csv"]
dfs = [pd.read_csv(filename) for filename in students_info]
df = pd.concat(dfs, ignore_index=True)
df["Nombre"] = df["Nombre"].str.strip().str.upper()
df["Apellido(s)"] = df["Apellido(s)"].str.strip().str.upper()
df["Grupos"] = df["Grupos"].astype(str).str.strip()

# Lista de nombres completos para autocompletar
nombres_completos = [
    f"{row['Apellido(s)']} {row['Nombre']}" for _, row in df.iterrows()
]
nombre_a_grupo = {
    f"{row['Apellido(s)']} {row['Nombre']}": row['Grupos'] for _, row in df.iterrows()
}

class JupyterExamReviewer:
    def __init__(self, examenes_dir="../data/examenes_procesados/"):
        self.examenes_dir = Path(examenes_dir)
        self.current_index = 0
        self.exam_files = []
        self.out = Output()
        self.status = Label(value="Cargando exámenes...")

        # Cargar todos los archivos PDF
        self._load_exam_files()

        if not self.exam_files:
            self.status.value = "No se encontraron exámenes para revisar"
            return

        # Configurar widgets
        self._setup_widgets()
        self._show_current_exam()

    def _load_exam_files(self):
        print("🔍 Escaneando carpetas de exámenes...")
        for grupo_dir in self.examenes_dir.iterdir():
            if grupo_dir.is_dir():
                for practica_dir in grupo_dir.iterdir():
                    if practica_dir.is_dir():
                        for pdf_file in practica_dir.glob("*.pdf"):
                            self.exam_files.append({
                                'file_path': pdf_file,
                                'carpeta_actual': grupo_dir.name,
                                'practica': practica_dir.name if practica_dir.name.startswith("Practica_") else "",
                                'nombre_archivo': pdf_file.stem
                            })
                # También incluir PDFs sueltos en la carpeta (por si acaso)
                for pdf_file in grupo_dir.glob("*.pdf"):
                    self.exam_files.append({
                        'file_path': pdf_file,
                        'carpeta_actual': grupo_dir.name,
                        'practica': "",
                        'nombre_archivo': pdf_file.stem
                    })
        self.exam_files.sort(key=lambda x: (x['carpeta_actual'], x['nombre_archivo']))
        print(f"📁 Total de exámenes encontrados: {len(self.exam_files)}")

    def _setup_widgets(self):
        # Status (ruta de archivo) - arriba
        self.status.layout = Layout(width='100%')
        
        # Nombre del alumno (principal)
        self.combo_nombre = Combobox(
            placeholder='Escribe o selecciona...',
            options=nombres_completos,
            description='Alumno:',
            layout=Layout(width='450px')
        )
        self.combo_nombre.observe(self._on_nombre_change, names='value')

        # Botón de aplicar cambios
        self.btn_apply = Button(
            description='✓ Aplicar',
            button_style='success',
            layout=Layout(width='80px')
        )
        
        # Botón para eliminar archivo (NUEVO)
        self.btn_delete = Button(
            description='🗑️ Eliminar',
            button_style='danger',
            layout=Layout(width='90px')
        )
        
        # Contador de progreso
        self.progress_label = Label(value="", layout=Layout(width='140px'))

        # Widgets de grupo, práctica y carpeta
        grupos_unicos = sorted(df["Grupos"].unique())
        self.dropdown_grupo = Dropdown(
            options=grupos_unicos,
            description='Grupo:',
            layout=Layout(width='280px')  # Más ancho para que se vea bien
        )

        self.dropdown_practica = Dropdown(
            options=['2', '3', '4', '5'],
            description='Práctica:',
            layout=Layout(width='180px')
        )

        # Selector de carpeta destino
        carpetas_disponibles = [d.name for d in self.examenes_dir.iterdir() if d.is_dir()]
        self.dropdown_carpeta = Dropdown(
            options=carpetas_disponibles,
            description='Carpeta:',
            layout=Layout(width='170px')
        )
        self.dropdown_carpeta.observe(self._on_carpeta_change, names='value')

        # HTML para mostrar sugerencias de nombres similares (horizontal)
        self.similares_html = HTML(value="", layout=Layout(width='100%', max_height='60px'))

        # Botones de navegación 
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='120px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='120px'))
        
        # Selector de página ajustado (botones a la derecha del texto)
        self.page_label = Label(value="Ver pág:", layout=Layout(width='50px'))
        self.page_selector = ToggleButtons(
            options=[('1', 1), ('2', 2)],
            value=1,
            description='',  # Quitamos la descripción del control y la ponemos separada
            style={'button_width': '30px'},
            layout=Layout(width='80px')
        )
        
        self.page_selector.observe(self._on_page_change, names='value')

        # Conectar eventos
        self.btn_prev.on_click(lambda x: self._navigate(-1))
        self.btn_next.on_click(lambda x: self._navigate(1))
        self.btn_apply.on_click(lambda x: self._apply_changes())
        self.btn_delete.on_click(lambda x: self._delete_current_file())  # NUEVO

        # REORGANIZACIÓN DE LA INTERFAZ:
        
        # Fila 1: Path/status (menos importante, arriba)
        status_row = HBox([self.status])
        
        # Fila 2: Nombre + aplicar + eliminar + progreso (MODIFICADO)
        nombre_row = HBox([
            self.combo_nombre,
            self.btn_apply,
            self.btn_delete,  # NUEVO
            self.progress_label
        ])
        
        # Fila 3: Grupo, práctica, carpeta (más visibles)
        opciones_row = HBox([
            self.dropdown_grupo,
            self.dropdown_practica, 
            self.dropdown_carpeta
        ])
        
        # Fila 4: Sugerencias similares (a lo ancho)
        sugerencias_row = HBox([self.similares_html])
        
        # Fila 5: Navegación (abajo) + selector de páginas en línea
        page_selector_group = HBox([
            self.page_label, 
            self.page_selector
            ], 
            layout=Layout(width='140px')
        )
        nav_row = HBox([
            self.btn_prev,
            self.btn_next,
            page_selector_group
        ])  

        # Layout general
        self.interface = VBox([
            status_row,
            nombre_row,
            opciones_row,
            sugerencias_row,
            nav_row,
            self.out
        ])

        display(self.interface)

    def _delete_current_file(self):
        """Elimina el archivo actual (NUEVO MÉTODO)"""
        if not self.exam_files:
            return
            
        current_exam = self.exam_files[self.current_index]
        file_path = current_exam['file_path']
        
        try:
            # Crear carpeta de eliminados si no existe
            deleted_folder = self.examenes_dir / "eliminados"
            deleted_folder.mkdir(exist_ok=True)
            
            # Mover a la carpeta de eliminados en lugar de eliminar permanentemente
            deleted_path = deleted_folder / file_path.name
            
            # Si ya existe en eliminados, añadir sufijo numérico
            contador = 2
            while deleted_path.exists():
                deleted_path = deleted_folder / f"{file_path.stem}_{contador}{file_path.suffix}"
                contador += 1
            
            # Mover archivo
            shutil.move(str(file_path), str(deleted_path))
            
            # Remover de la lista
            self.exam_files.pop(self.current_index)
            
            # Ajustar índice si es necesario
            if self.current_index >= len(self.exam_files):
                self.current_index = max(0, len(self.exam_files) - 1)
            
            self.status.value = f"🗑️ Eliminado: {file_path.name} → eliminados/{deleted_path.name}"
            
            # Mostrar siguiente examen o mensaje si no hay más
            if self.exam_files:
                self._show_current_exam()
            else:
                self.status.value = "🎉 No hay más exámenes para revisar"
                with self.out:
                    clear_output(wait=True)
                    print("No hay más exámenes para revisar")
                    
        except Exception as e:
            self.status.value = f"❌ Error eliminando archivo: {e}"

    def _on_carpeta_change(self, change):
        # Eliminado: Ya no se deshabilita el dropdown de práctica para ninguna carpeta
        pass

    def _on_nombre_change(self, change):
        valor = change['new']
        if valor in nombre_a_grupo:
            self.dropdown_grupo.value = nombre_a_grupo[valor]
            self.combo_nombre.value = valor
        self._update_similares(valor)

    def _on_page_change(self, change):
        self._show_exam_image(self.exam_files[self.current_index]['file_path'])

    def _update_similares(self, valor):
        if valor:
            matches = get_close_matches(valor, nombres_completos, n=4, cutoff=0)
            # Mostrar horizontalmente (una sola fila)
            html = "<b>Sugerencias:</b> "
            if matches:
                html += "<span style='display:inline-block;white-space:nowrap;'>"
                for i, m in enumerate(matches):
                    html += f"<span style='display:inline-block;margin-right:20px;font-size:90%'>{m} <span style='color:#888;font-size:85%'>({nombre_a_grupo.get(m, '-')})</span></span>"
                html += "</span>"
            else:
                html += "<i>No hay sugerencias</i>"
            self.similares_html.value = html
        else:
            self.similares_html.value = ""

    def _show_current_exam(self):
        if not self.exam_files:
            return

        current_exam = self.exam_files[self.current_index]
        self.progress_label.value = f"Examen {self.current_index + 1} de {len(self.exam_files)}"

        nombre_completo = current_exam['nombre_archivo'].replace('_', ' ')
        mejor_match = None
        for n in nombres_completos:
            if nombre_completo.upper() in n.upper():
                mejor_match = n
                break
        if mejor_match:
            self.combo_nombre.value = mejor_match
            self.dropdown_grupo.value = nombre_a_grupo[mejor_match]
        else:
            self.combo_nombre.value = nombre_completo
            self.dropdown_grupo.value = df["Grupos"].iloc[0]  # valor por defecto

        self._update_similares(self.combo_nombre.value)

        # Práctica
        if current_exam['practica']:
            practica_num = current_exam['practica'].replace('Practica_', '')
            if practica_num in ['2', '3', '4', '5']:
                self.dropdown_practica.value = practica_num
            else:
                self.dropdown_practica.value = '3'
        else:
            self.dropdown_practica.value = '3'

        # Carpeta actual
        self.dropdown_carpeta.value = current_exam['carpeta_actual']

        # Mostrar solo la página 1 por defecto
        self.page_selector.value = 1

        # Ajustar el rango del selector de página según el número de páginas
        reader = PdfReader(current_exam['file_path'])
        num_pages = len(reader.pages)
        if num_pages == 1:
            self.dropdown_carpeta.value = "problemático"
            self.page_selector.disabled = True
        else:
            self.page_selector.disabled = False

        # Eliminado: Ya no se deshabilita el dropdown de práctica

        if current_exam['practica']:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['practica']} / {current_exam['nombre_archivo']}.pdf"
        else:
            self.status.value = f"📁 {current_exam['carpeta_actual']} / {current_exam['nombre_archivo']}.pdf"

        self._show_exam_image(current_exam['file_path'])

    def _show_exam_image(self, pdf_path):
        with self.out:
            clear_output(wait=True)
            try:
                reader = PdfReader(pdf_path)
                num_pages = len(reader.pages)
                page_num = self.page_selector.value
                if num_pages == 1:
                    page_num = 1
                images = convert_from_path(
                    pdf_path,
                    first_page=page_num,
                    last_page=page_num,
                    dpi=150,
                    fmt='jpeg'
                )
                if images:
                    image = images[0]
                    # Recortar más agresivamente por arriba y por abajo (1/4 de la altura)
                    cabecera_altura = int(min(450, image.height // 4))
                    # Recortar un poco desde arriba también (20 píxeles)
                    top_offset = 20
                    cropped = image.crop((0, top_offset, image.width, cabecera_altura))
                    plt.figure(figsize=(10, 6))  # Altura reducida
                    plt.imshow(cropped)
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                else:
                    print("❌ No se pudo cargar la imagen del PDF")
            except Exception as e:
                print(f"❌ Error cargando imagen: {e}")

    def _navigate(self, direction):
        if self._has_pending_changes():
            self._apply_changes()
        new_index = self.current_index + direction
        if 0 <= new_index < len(self.exam_files):
            self.current_index = new_index
            self._show_current_exam()
        elif new_index >= len(self.exam_files):
            self.status.value = "🎉 ¡Revisión completada! Has llegado al final."
        elif new_index < 0:
            self.status.value = "📍 Ya estás en el primer examen."

    def _has_pending_changes(self):
        if not self.exam_files:
            return False
        current_exam = self.exam_files[self.current_index]
        nombre_actual = self.combo_nombre.value.strip().upper().replace(' ', '_')
        nombre_archivo_actual = current_exam['nombre_archivo'].upper()
        carpeta_cambio = self.dropdown_carpeta.value != current_exam['carpeta_actual']
        practica_actual = current_exam['practica'].replace('Practica_', '') if current_exam['practica'] else ''
        # Eliminado: "and not self.dropdown_practica.disabled" ya que nunca se deshabilita
        practica_cambio = self.dropdown_practica.value != practica_actual
        grupo_cambio = self.dropdown_grupo.value != nombre_a_grupo.get(self.combo_nombre.value, self.dropdown_grupo.value)
        nombre_cambio = nombre_actual != nombre_archivo_actual
        return carpeta_cambio or practica_cambio or grupo_cambio or nombre_cambio

    def _apply_changes(self):
        if not self.exam_files:
            return
        current_exam = self.exam_files[self.current_index]
        old_path = current_exam['file_path']
        try:
            nombre_nuevo = self.combo_nombre.value.strip().replace(' ', '_')
            grupo_para_nombre = self.dropdown_grupo.value
            nueva_practica = self.dropdown_practica.value
            carpeta_destino = self.dropdown_carpeta.value

            # Nombre base: APELLIDOS_NOMBRE_P<num practica>_<Grupo>.pdf
            nombre_base = f"{nombre_nuevo}_P{nueva_practica}_{grupo_para_nombre}"
            nueva_carpeta = self.examenes_dir / carpeta_destino
            if carpeta_destino not in ["extraviados", "problemático"]:
                nueva_carpeta = nueva_carpeta / f"Practica_{nueva_practica}"
            nueva_carpeta.mkdir(parents=True, exist_ok=True)
            
            # COMPROBACIÓN MEJORADA DE ARCHIVOS EXISTENTES
            nuevo_path = nueva_carpeta / f"{nombre_base}.pdf"
            
            # Si el archivo destino es el mismo que el origen, no hacer nada
            if nuevo_path == old_path:
                self.status.value = "ℹ️ Sin cambios necesarios"
                return
            
            # Si existe otro archivo con el mismo nombre, buscar sufijo disponible
            contador = 2
            while nuevo_path.exists():
                # Extraer cualquier sufijo numérico existente del nombre base
                match = re.search(r'_(\d+)$', nombre_base)
                if match:
                    # Ya tiene sufijo numérico, incrementarlo
                    numero_actual = int(match.group(1))
                    nombre_sin_sufijo = nombre_base[:match.start()]
                    nuevo_nombre_base = f"{nombre_sin_sufijo}_{numero_actual + contador - 1}"
                else:
                    # No tiene sufijo, añadir uno
                    nuevo_nombre_base = f"{nombre_base}_{contador}"
                
                nuevo_path = nueva_carpeta / f"{nuevo_nombre_base}.pdf"
                contador += 1
                
                # Seguridad: evitar bucle infinito
                if contador > 100:
                    self.status.value = "❌ Error: Demasiados archivos duplicados"
                    return
            
            # Realizar el movimiento de archivo
            try:
                shutil.move(str(old_path), str(nuevo_path))
                
                # Actualizar información en la lista
                current_exam['file_path'] = nuevo_path
                current_exam['carpeta_actual'] = carpeta_destino
                current_exam['practica'] = f"Practica_{nueva_practica}" if carpeta_destino not in ["extraviados", "problemático"] else ""
                current_exam['nombre_archivo'] = nuevo_path.stem
                
                # Mensaje de confirmación
                if carpeta_destino in ["extraviados", "problemático"]:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/{nuevo_path.name}"
                else:
                    self.status.value = f"✅ Movido a: {carpeta_destino}/Practica_{nueva_practica}/{nuevo_path.name}"
                    
                # Si se añadió un sufijo, avisar
                if contador > 2:
                    self.status.value += f" (duplicado evitado con sufijo _{contador-2})"
                    
            except PermissionError:
                self.status.value = "❌ Error: Archivo en uso o sin permisos"
            except FileNotFoundError:
                self.status.value = "❌ Error: Archivo origen no encontrado"
            except Exception as move_error:
                self.status.value = f"❌ Error moviendo archivo: {move_error}"
                
        except Exception as e:
            self.status.value = f"❌ Error aplicando cambios: {e}"

def iniciar_revision_examenes(examenes_dir="../data/examenes_procesados/"):
    print("🚀 Iniciando revisor de exámenes...")
    reviewer = JupyterExamReviewer(examenes_dir)
    return reviewer

In [142]:
# Para lanzar la interfaz:
#reviewer = iniciar_revision_examenes()

#### Backup para asegurar...

In [35]:
import zipfile
from datetime import datetime  # AÑADIR ESTE IMPORT

def crear_backup_examenes(carpeta_examenes="../data/examenes_procesados/", carpeta_backup="../data/"):
    """
    Crea un backup completo de la carpeta de exámenes procesados
    """
    carpeta_examenes = Path(carpeta_examenes)
    carpeta_backup = Path(carpeta_backup)
    
    # Crear nombre del backup con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_backup = f"examenes_procesados_backup_{timestamp}.zip"
    ruta_backup = carpeta_backup / nombre_backup
    
    print(f"🔄 Creando backup de: {carpeta_examenes}")
    print(f"📦 Archivo de backup: {ruta_backup}")
    
    try:
        # Crear el archivo ZIP
        with zipfile.ZipFile(ruta_backup, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Recorrer todos los archivos y carpetas
            for root, dirs, files in os.walk(carpeta_examenes):
                for file in files:
                    file_path = Path(root) / file
                    # Calcular la ruta relativa para mantener la estructura
                    arcname = file_path.relative_to(carpeta_examenes.parent)
                    zipf.write(file_path, arcname)
                    print(f"  ✓ Añadido: {arcname}")
        
        print(f"\n✅ Backup completado exitosamente!")
        print(f"📁 Tamaño del backup: {ruta_backup.stat().st_size / (1024*1024):.2f} MB")
        print(f"💾 Ubicación: {ruta_backup}")
        
        return str(ruta_backup)
        
    except Exception as e:
        print(f"❌ Error creando backup: {e}")
        return None

In [ ]:
#crear_backup_examenes()

In [36]:
import os
from pathlib import Path

def eliminar_carpetas_vacias(ruta_base="../data/examenes_procesados/"):
    """
    Elimina recursivamente todas las carpetas vacías en la ruta especificada
    """
    ruta_base = Path(ruta_base)
    
    if not ruta_base.exists():
        print(f"❌ La ruta {ruta_base} no existe")
        return
    
    carpetas_eliminadas = []
    
    # Función recursiva para eliminar carpetas vacías
    def eliminar_vacias_recursivo(directorio):
        """Elimina carpetas vacías de forma recursiva, empezando por las más profundas"""
        try:
            # Primero procesar subdirectorios
            for item in directorio.iterdir():
                if item.is_dir():
                    eliminar_vacias_recursivo(item)
            
            # Luego verificar si el directorio actual está vacío
            if directorio.is_dir() and not any(directorio.iterdir()):
                directorio.rmdir()
                carpetas_eliminadas.append(str(directorio))
                print(f"🗑️ Eliminada carpeta vacía: {directorio}")
                
        except PermissionError:
            print(f"⚠️ Sin permisos para eliminar: {directorio}")
        except OSError as e:
            print(f"⚠️ Error eliminando {directorio}: {e}")
    
    print(f"🔍 Buscando carpetas vacías en: {ruta_base}")
    print("=" * 50)
    
    # Ejecutar la eliminación recursiva
    eliminar_vacias_recursivo(ruta_base)
    
    print("=" * 50)
    print(f"✅ Proceso completado")
    print(f"📊 Total de carpetas vacías eliminadas: {len(carpetas_eliminadas)}")
    
    if carpetas_eliminadas:
        print("\n📋 Carpetas eliminadas:")
        for carpeta in carpetas_eliminadas:
            print(f"  - {carpeta}")
    else:
        print("ℹ️ No se encontraron carpetas vacías para eliminar")
    
    return carpetas_eliminadas


In [ ]:
#eliminar_carpetas_vacias()

### Buscar practica concreta por si alguien se ha equivocado de montón

In [37]:
def buscar_alumnos_practicas(
        ruta_examenes="../data/examenes_procesados/",
        practicas=["3", "5"]  # Usar strings por defecto
):
    """
    Busca alumnos que han entregado prácticas basándose en la estructura de carpetas
    y nombres de archivos. Marca con * los que están en carpeta problemático.
    
    Args:
        ruta_examenes: Ruta a la carpeta donde están los exámenes procesados
        practicas: Lista de números de práctica como strings ["3", "5"]
    """
    # Asegurar que los elementos de practicas son strings
    practicas = [str(p) for p in practicas]
    
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"❌ La ruta {ruta_examenes} no existe")
        return []
    
    alumnos_practicas = []
    
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Procesar carpetas de grupos normales
        if carpeta_grupo.name not in ["extraviados", "eliminados"]:
            # Si es problemático, marcar con asterisco
            asterisco = " *" if carpeta_grupo.name == "problemático" else ""
            
            if carpeta_grupo.name == "problemático":
                # En problemático, buscar directamente archivos PDF
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica in practicas:
                        info_alumno = extraer_info_alumno(archivo_pdf, practica, "problemático", True)
                        if info_alumno:
                            alumnos_practicas.append(info_alumno)
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        
                        if practica_num in practicas:
                            print(f"📂 Encontrada {subcarpeta.name} en grupo {carpeta_grupo.name}")
                            
                            for archivo_pdf in subcarpeta.glob("*.pdf"):
                                info_alumno = extraer_info_alumno(archivo_pdf, practica_num, carpeta_grupo.name, False)
                                if info_alumno:
                                    alumnos_practicas.append(info_alumno)
    
    print("\n" + "=" * 60)
    print(f"📊 RESUMEN: Encontrados {len(alumnos_practicas)} alumnos con prácticas "+str(practicas))
    
    if alumnos_practicas:
        # Organizar por grupo y práctica
        por_grupo_practica = {}
        
        for alumno in alumnos_practicas:
            grupo = alumno['grupo']
            practica = alumno['practica']
            key = f"{grupo}_P{practica}"
            
            if key not in por_grupo_practica:
                por_grupo_practica[key] = []
            por_grupo_practica[key].append(alumno)
        
        print(f"\n📋 LISTADO POR GRUPO Y PRÁCTICA:")
        print("=" * 60)
        
        for key in sorted(por_grupo_practica.keys()):
            grupo, practica_info = key.split('_P')
            asterisco = " *" if any(a['problematico'] for a in por_grupo_practica[key]) else ""
            
            print(f"\n📚 GRUPO {grupo} - PRÁCTICA {practica_info}{asterisco}")
            print("-" * 40)
            
            for alumno in sorted(por_grupo_practica[key], key=lambda x: (x['apellidos'], x['nombre'])):
                asterisco_individual = " *" if alumno['problematico'] else ""
                print(f"  • {alumno['apellidos']}, {alumno['nombre']}{asterisco_individual}")
    
    return alumnos_practicas

def extraer_info_alumno(archivo_pdf, practica, grupo, es_problematico):
    """Extrae información del alumno desde el nombre del archivo"""
    nombre_archivo = archivo_pdf.stem
    
    # Limpiar el nombre del archivo para extraer apellidos y nombre
    nombre_limpio = nombre_archivo
    
    # Remover elementos conocidos (práctica, grupo, etc.)
    patrones_a_remover = [
        rf'_P{practica}', rf'P{practica}_', rf'P{practica}$',
        rf'_{grupo}', rf'{grupo}_', rf'^{grupo}',
        r'_CITIM\d+', r'_IWSIM\d+', r'_CITIT\d+', r'_IWSIT\d+',
        r'CITIM\d+_', r'IWSIM\d+_', r'CITIT\d+_', r'IWSIT\d+_',
        r'_\d+$'  # Números al final
    ]
    
    for patron in patrones_a_remover:
        nombre_limpio = re.sub(patron, '', nombre_limpio, flags=re.IGNORECASE)
    
    # Limpiar guiones bajos múltiples
    nombre_limpio = re.sub(r'_+', '_', nombre_limpio).strip('_')
    
    # Separar apellidos y nombre
    partes = nombre_limpio.split('_')
    
    if len(partes) >= 2:
        # Asumir que la última parte es el nombre y el resto apellidos
        apellidos = '_'.join(partes[:-1]).replace('_', ' ')
        nombre = partes[-1]
    elif len(partes) == 1:
        apellidos = partes[0]
        nombre = ''
    else:
        apellidos = nombre_archivo
        nombre = ''
    
    return {
        'archivo': archivo_pdf.name,
        'apellidos': apellidos,
        'nombre': nombre,
        'practica': practica,
        'grupo': grupo,
        'problematico': es_problematico
    }

def detectar_practica_del_nombre(nombre_archivo):
    """Detecta el número de práctica del nombre del archivo"""
    nombre = nombre_archivo.upper()
    
    # Buscar patrones de práctica
    patrones = [
        r'P(\d)',
        r'_(\d)_',
        r'PRACTICA_?(\d)',
    ]
    
    for patron in patrones:
        match = re.search(patron, nombre)
        if match:
            return match.group(1)
    
    return None

In [ ]:
#buscar_alumnos_practicas(practicas=[2,4])

### Buscar examenes de alumnos por grupo

In [38]:

def detectar_grupo_del_nombre(nombre_archivo):
    """Detecta el grupo del nombre del archivo"""
    nombre = nombre_archivo.upper()
    
    # Buscar patrones de grupo
    grupos_posibles = ['CITIM11', 'CITIM12', 'IWSIM11', 'IWSIM12', 'CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12']
    
    for grupo in grupos_posibles:
        if grupo in nombre:
            return grupo
    
    return None

In [39]:
def extraer_info_alumno(archivo_pdf, practica, grupo, es_problematico):
    """
    Extrae información del alumno desde el nombre del archivo para grupos CITIT/IWSIT
    """
    nombre_archivo = archivo_pdf.stem
    
    # Limpiar el nombre del archivo para extraer apellidos y nombre
    nombre_limpio = nombre_archivo
    
    # Remover elementos conocidos (práctica, grupo, etc.)
    patrones_a_remover = [
        rf'_P{practica}', rf'P{practica}_', rf'P{practica}$',
        rf'_{grupo}', rf'{grupo}_', rf'^{grupo}',
        r'_CITIT\d+', r'_IWSIT\d+', r'_CITIM\d+', r'_IWSIM\d+',
        r'CITIT\d+_', r'IWSIT\d+_', r'CITIM\d+_', r'IWSIM\d+_',
        r'_\d+$'  # Números al final
    ]
    
    for patron in patrones_a_remover:
        nombre_limpio = re.sub(patron, '', nombre_limpio, flags=re.IGNORECASE)
    
    # Limpiar guiones bajos múltiples
    nombre_limpio = re.sub(r'_+', '_', nombre_limpio).strip('_')
    
    # Separar apellidos y nombre
    partes = nombre_limpio.split('_')
    
    if len(partes) >= 2:
        # Asumir que la última parte es el nombre y el resto apellidos
        apellidos = '_'.join(partes[:-1]).replace('_', ' ')
        nombre = partes[-1]
    elif len(partes) == 1:
        apellidos = partes[0]
        nombre = ''
    else:
        apellidos = nombre_archivo
        nombre = ''
    
    return {
        'archivo': archivo_pdf.name,
        'apellidos': apellidos,
        'nombre': nombre,
        'practica': practica,
        'grupo': grupo,
        'problematico': es_problematico
    }

def buscar_alumnos_grupos_citit_iwsit(ruta_examenes="../data/examenes_procesados/"):
    """
    Busca alumnos de los grupos CITIT11, CITIT12, IWSIT11, IWSIT12 clasificados por práctica.
    Verifica el grupo correcto consultando el DataFrame de alumnos.
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes} no existe")
        return []
    
    # Grupos objetivo
    grupos_objetivo = ['CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12']
    alumnos_encontrados = []
    
    print("Buscando alumnos de grupos CITIT/IWSIT...")
    print("=" * 60)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Verificar si es uno de los grupos objetivo O carpeta problemático
        es_grupo_objetivo = carpeta_grupo.name in grupos_objetivo
        es_problematico = carpeta_grupo.name == "problemático"
        
        if es_grupo_objetivo or es_problematico:
            print(f"\nProcesando carpeta: {carpeta_grupo.name}")
            
            if es_problematico:
                # En problemático, buscar directamente archivos PDF y verificar si son de grupos objetivo
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    grupo_detectado = detectar_grupo_del_nombre(archivo_pdf.name)
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica:
                        info_alumno = extraer_info_alumno(archivo_pdf, practica, grupo_detectado or carpeta_grupo.name, True)
                        if info_alumno:
                            # Verificar grupo real en el DataFrame
                            grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                            if grupo_real in grupos_objetivo:
                                info_alumno['grupo_real'] = grupo_real
                                info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                alumnos_encontrados.append(info_alumno)
                                print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (P{practica}, {grupo_real}) *")
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        print(f"  {subcarpeta.name}")
                        
                        for archivo_pdf in subcarpeta.glob("*.pdf"):
                            info_alumno = extraer_info_alumno(archivo_pdf, practica_num, carpeta_grupo.name, False)
                            if info_alumno:
                                # Verificar grupo real en el DataFrame
                                grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                                if grupo_real in grupos_objetivo:
                                    info_alumno['grupo_real'] = grupo_real
                                    info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                    alumnos_encontrados.append(info_alumno)
                                    
                                    # Mostrar advertencia si el grupo de carpeta no coincide con el real
                                    if grupo_real != carpeta_grupo.name:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (Grupo real: {grupo_real}, Carpeta: {carpeta_grupo.name}) ⚠️")
                                    else:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']}")
    
    print("\n" + "=" * 60)
    print(f"RESUMEN: Encontrados {len(alumnos_encontrados)} alumnos de grupos CITIT/IWSIT")
    
    if alumnos_encontrados:
        # Organizar por práctica y luego por grupo REAL
        por_practica = {}
        
        for alumno in alumnos_encontrados:
            practica = alumno['practica']
            if practica not in por_practica:
                por_practica[practica] = {}
            
            grupo = alumno['grupo_real']  # Usar grupo real del DataFrame
            if grupo not in por_practica[practica]:
                por_practica[practica][grupo] = []
            
            por_practica[practica][grupo].append(alumno)
        
        print(f"\nLISTADO SEPARADO EN DOS LISTAS:")
        print("=" * 80)
        
        # Ordenar por número de práctica
        for practica in sorted(por_practica.keys(), key=lambda x: int(x) if x.isdigit() else 999):
            print(f"\nPractica {practica}:")
            print("-" * 50)
            
            # Ordenar grupos alfabéticamente
            for grupo in sorted(por_practica[practica].keys()):
                alumnos_grupo = por_practica[practica][grupo]
                
                # Separar en problemáticos y normales
                alumnos_normales = [alumno for alumno in alumnos_grupo if not alumno['problematico']]
                alumnos_problematicos = [alumno for alumno in alumnos_grupo if alumno['problematico']]
                
                print(f"\nGrupo {grupo}:")
                
                # Lista de alumnos normales
                if alumnos_normales:
                    print("  Alumnos normales:")
                    for alumno in sorted(alumnos_normales, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']}")
                
                # Lista de alumnos problemáticos (dentro del mismo grupo)
                if alumnos_problematicos:
                    print("  Alumnos problemáticos:")
                    for alumno in sorted(alumnos_problematicos, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']} *")
        
        print(f"\n" + "-" * 40)
        print(f"RESUMEN POR GRUPO:")
        print("-" * 40)
        
        resumen_grupos = {}
        for alumno in alumnos_encontrados:
            grupo = alumno['grupo_real']  # Usar grupo real
            if grupo not in resumen_grupos:
                resumen_grupos[grupo] = {'total': 0, 'problematicos': 0}
            resumen_grupos[grupo]['total'] += 1
            if alumno['problematico']:
                resumen_grupos[grupo]['problematicos'] += 1
        
        for grupo in sorted(resumen_grupos.keys()):
            total = resumen_grupos[grupo]['total']
            problematicos = resumen_grupos[grupo]['problematicos']
            print(f"  {grupo}: {total} alumnos (problemáticos: {problematicos})")
    
    return alumnos_encontrados

def verificar_grupo_en_dataframe(apellidos, nombre):
    """
    Verifica el grupo real del alumno consultando el DataFrame df
    """
    if 'df' not in globals():
        print("⚠️ DataFrame df no está disponible")
        return "GRUPO_NO_ENCONTRADO"
    
    apellidos_limpio = apellidos.upper().strip()
    nombre_limpio = nombre.upper().strip()
    
    # Buscar coincidencia exacta primero
    mask_exacta = (df['Apellido(s)'].str.upper().str.strip() == apellidos_limpio) & \
                  (df['Nombre'].str.upper().str.strip() == nombre_limpio)
    
    if mask_exacta.any():
        return df.loc[mask_exacta, 'Grupos'].iloc[0]
    
    # Si no hay coincidencia exacta, buscar coincidencia parcial
    if apellidos_limpio and nombre_limpio:
        mask_parcial = df['Apellido(s)'].str.upper().str.contains(apellidos_limpio[:5], na=False) & \
                       df['Nombre'].str.upper().str.contains(nombre_limpio[:3], na=False)
        
        if mask_parcial.any():
            return df.loc[mask_parcial, 'Grupos'].iloc[0]
    
    return "GRUPO_NO_ENCONTRADO"

In [40]:
def buscar_alumnos_grupos(df, ruta_examenes="../data/examenes_procesados/", grupos_objetivo = ['CITIT11', 'CITIT12', 'IWSIT11', 'IWSIT12'] ):
    """
    Busca alumnos de los grupos CITIT11, CITIT12, IWSIT11, IWSIT12 clasificados por práctica.
    Verifica el grupo correcto consultando el DataFrame de alumnos.
    """
    ruta_examenes = Path(ruta_examenes)
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes} no existe")
        return []
    
    
    alumnos_encontrados = []
    
    print("Buscando alumnos de grupos " + str(grupos_objetivo))
    print("=" * 60)
    
    # Recorrer todas las carpetas de grupos
    for carpeta_grupo in ruta_examenes.iterdir():
        if not carpeta_grupo.is_dir():
            continue
            
        # Verificar si es uno de los grupos objetivo O carpeta problemático
        es_grupo_objetivo = carpeta_grupo.name in grupos_objetivo
        es_problematico = carpeta_grupo.name == "problemático"
        
        if es_grupo_objetivo or es_problematico:
            print(f"\nProcesando carpeta: {carpeta_grupo.name}")
            
            if es_problematico:
                # En problemático, buscar directamente archivos PDF y verificar si son de grupos objetivo
                for archivo_pdf in carpeta_grupo.glob("*.pdf"):
                    grupo_detectado = detectar_grupo_del_nombre(archivo_pdf.name)
                    practica = detectar_practica_del_nombre(archivo_pdf.name)
                    if practica:
                        info_alumno = extraer_info_alumno(archivo_pdf, practica, grupo_detectado or carpeta_grupo.name, True)
                        if info_alumno:
                            # Verificar grupo real en el DataFrame
                            grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                            if grupo_real in grupos_objetivo:
                                info_alumno['grupo_real'] = grupo_real
                                info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                alumnos_encontrados.append(info_alumno)
                                print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (P{practica}, {grupo_real}) *")
            else:
                # Buscar en subcarpetas de prácticas
                for subcarpeta in carpeta_grupo.iterdir():
                    if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                        practica_num = subcarpeta.name.replace("Practica_", "")
                        print(f"  {subcarpeta.name}")
                        
                        for archivo_pdf in subcarpeta.glob("*.pdf"):
                            info_alumno = extraer_info_alumno(archivo_pdf, practica_num, carpeta_grupo.name, False)
                            if info_alumno:
                                # Verificar grupo real en el DataFrame
                                grupo_real = verificar_grupo_en_dataframe(info_alumno['apellidos'], info_alumno['nombre'])
                                if grupo_real in grupos_objetivo:
                                    info_alumno['grupo_real'] = grupo_real
                                    info_alumno['grupo_carpeta'] = carpeta_grupo.name
                                    alumnos_encontrados.append(info_alumno)
                                    
                                    # Mostrar advertencia si el grupo de carpeta no coincide con el real
                                    if grupo_real != carpeta_grupo.name:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']} (Grupo real: {grupo_real}, Carpeta: {carpeta_grupo.name}) ⚠️")
                                    else:
                                        print(f"    {info_alumno['apellidos']}, {info_alumno['nombre']}")
    
    print("\n" + "=" * 60)
    print(f"RESUMEN: Encontrados {len(alumnos_encontrados)} alumnos de grupos CITIT/IWSIT")
    
    if alumnos_encontrados:
        # Organizar por práctica y luego por grupo REAL
        por_practica = {}
        
        for alumno in alumnos_encontrados:
            practica = alumno['practica']
            if practica not in por_practica:
                por_practica[practica] = {}
            
            grupo = alumno['grupo_real']  # Usar grupo real del DataFrame
            if grupo not in por_practica[practica]:
                por_practica[practica][grupo] = []
            
            por_practica[practica][grupo].append(alumno)
        
        print(f"\nLISTADO SEPARADO EN DOS LISTAS:")
        print("=" * 80)
        
        # Ordenar por número de práctica
        for practica in sorted(por_practica.keys(), key=lambda x: int(x) if x.isdigit() else 999):
            print(f"\nPractica {practica}:")
            print("-" * 50)
            
            # Ordenar grupos alfabéticamente
            for grupo in sorted(por_practica[practica].keys()):
                alumnos_grupo = por_practica[practica][grupo]
                
                # Separar en problemáticos y normales
                alumnos_normales = [alumno for alumno in alumnos_grupo if not alumno['problematico']]
                alumnos_problematicos = [alumno for alumno in alumnos_grupo if alumno['problematico']]
                
                print(f"\nGrupo {grupo}:")
                
                # Lista de alumnos normales
                if alumnos_normales:
                    print("  Alumnos normales:")
                    for alumno in sorted(alumnos_normales, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']}")
                
                # Lista de alumnos problemáticos (dentro del mismo grupo)
                if alumnos_problematicos:
                    print("  Alumnos problemáticos:")
                    for alumno in sorted(alumnos_problematicos, key=lambda x: (x['apellidos'], x['nombre'])):
                        print(f"    {alumno['apellidos']}, {alumno['nombre']} *")
        
        print(f"\n" + "-" * 40)
        print(f"RESUMEN POR GRUPO:")
        print("-" * 40)
        
        resumen_grupos = {}
        for alumno in alumnos_encontrados:
            grupo = alumno['grupo_real']  # Usar grupo real
            if grupo not in resumen_grupos:
                resumen_grupos[grupo] = {'total': 0, 'problematicos': 0}
            resumen_grupos[grupo]['total'] += 1
            if alumno['problematico']:
                resumen_grupos[grupo]['problematicos'] += 1
        
        for grupo in sorted(resumen_grupos.keys()):
            total = resumen_grupos[grupo]['total']
            problematicos = resumen_grupos[grupo]['problematicos']
            print(f"  {grupo}: {total} alumnos (problemáticos: {problematicos})")
    
    return alumnos_encontrados

def verificar_grupo_en_dataframe(apellidos, nombre):
    """
    Verifica el grupo real del alumno consultando el DataFrame df
    """
    if 'df' not in globals():
        print("⚠️ DataFrame df no está disponible")
        return "GRUPO_NO_ENCONTRADO"
    
    apellidos_limpio = apellidos.upper().strip()
    nombre_limpio = nombre.upper().strip()
    
    # Buscar coincidencia exacta primero
    mask_exacta = (df['Apellido(s)'].str.upper().str.strip() == apellidos_limpio) & \
                  (df['Nombre'].str.upper().str.strip() == nombre_limpio)
    
    if mask_exacta.any():
        return df.loc[mask_exacta, 'Grupos'].iloc[0]
    
    # Si no hay coincidencia exacta, buscar coincidencia parcial
    if apellidos_limpio and nombre_limpio:
        mask_parcial = df['Apellido(s)'].str.upper().str.contains(apellidos_limpio[:5], na=False) & \
                       df['Nombre'].str.upper().str.contains(nombre_limpio[:3], na=False)
        
        if mask_parcial.any():
            return df.loc[mask_parcial, 'Grupos'].iloc[0]
    
    return "GRUPO_NO_ENCONTRADO"

In [41]:
alumnos_citit_iwsit = buscar_alumnos_grupos(
    df_students, grupos_objetivo=['CITIT12']
)

Buscando alumnos de grupos ['CITIT12']

RESUMEN: Encontrados 0 alumnos de grupos CITIT/IWSIT


### Analizar origen ficheros en 'problematico' (pdf original) 
Objetivo: ver en los físicos si tienen hoja en blanco o ha sido error de la impresora


In [42]:
import tqdm 
def analizar_examenes_problematicos(ruta_examenes="../data/examenes_procesados/", 
                                   ruta_saved="../data/saved/", 
                                   parar_en_100=False,
                                   umbral_similitud=0.85):
    """
    Analiza todos los exámenes en la carpeta 'problemático' comparando SOLO con saved.
    Muestra los 3 mejores resultados por alumno en tiempo real.
    USA DIRECTAMENTE comparar_pdfs_visuales_con_tqdm - versión simplificada
    """
    
    ruta_problematico = Path(ruta_examenes) / "problemático"
    
    if not ruta_problematico.exists():
        print("❌ No existe la carpeta 'problemático'")
        return pd.DataFrame()
    
    archivos_problematicos = list(ruta_problematico.glob("*.pdf"))
    
    if not archivos_problematicos:
        print("ℹ️ No hay archivos en la carpeta 'problemático'")
        return pd.DataFrame()
    
    print(f"🔍 Analizando {len(archivos_problematicos)} archivos problemáticos...")
    print("💾 Solo comparando con archivos en /saved")
    if parar_en_100:
        print("⏹️ Modo parada automática activado (se detiene al encontrar 100% similitud)")
    
    resultados = []
    
    # Barra de progreso principal con tqdm
    for archivo_objetivo in tqdm(archivos_problematicos, desc="📄 Procesando archivos", unit="archivo"):
        try:
            # Mostrar archivo actual
            tqdm.write(f"\n📄 Procesando: {archivo_objetivo.name}")
            
            # USAR DIRECTAMENTE comparar_pdfs_visuales_con_tqdm con ruta_lotes vacía
            # para que SOLO compare con /saved
            coincidencias = comparar_pdfs_visuales_con_tqdm_solo_saved(
                archivo_objetivo.name,  # Solo el nombre del archivo
                ruta_saved=ruta_saved,
                ruta_examenes=str(ruta_problematico),  # Buscar en problemático
                mostrar_detalles=False,  # Silencioso para no saturar output
                parar_en_100=parar_en_100,
                umbral_similitud=umbral_similitud
            )
            
            if coincidencias:
                # Filtrar solo coincidencias de tipo 'saved' y ordenar por similitud
                coincidencias_saved = [c for c in coincidencias if c.get('tipo') == 'saved']
                mejores_coincidencias = sorted(coincidencias_saved, key=lambda x: x['similitud'], reverse=True)[:3]
                
                # Agregar información adicional a cada coincidencia
                for coincidencia in mejores_coincidencias:
                    coincidencia['archivo_problematico'] = archivo_objetivo.name
                
                resultados.extend(mejores_coincidencias)
                
                # MOSTRAR RESULTADOS EN TIEMPO REAL
                tqdm.write("🎯 MEJORES COINCIDENCIAS ENCONTRADAS:")
                tqdm.write("-" * 60)
                for j, coincidencia in enumerate(mejores_coincidencias, 1):
                    icono = "🥇" if j == 1 else "🥈" if j == 2 else "🥉"
                    
                    tqdm.write(f"{icono} #{j} 💾 {coincidencia['similitud']:.1%} - {coincidencia['archivo']}")
                    tqdm.write(f"    📄 Páginas: {coincidencia['paginas']}")
                        
                    if parar_en_100 and coincidencia['similitud'] >= 0.999:
                        tqdm.write("    🎯 ¡COINCIDENCIA PERFECTA! Búsqueda detenida.")
                        break
                        
            else:
                # Sin coincidencias
                tqdm.write("❌ Sin coincidencias encontradas")
                reader_objetivo = PdfReader(archivo_objetivo)
                num_paginas_objetivo = len(reader_objetivo.pages)
                
                resultados.append({
                    'archivo_problematico': archivo_objetivo.name,
                    'archivo': 'SIN_COINCIDENCIAS',
                    'ruta': '',
                    'tipo': 'ninguno',
                    'similitud': 0.0,
                    'paginas': num_paginas_objetivo
                })
                
        except Exception as e:
            tqdm.write(f"❌ Error procesando {archivo_objetivo.name}: {e}")
            continue
    
    # Crear DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    if df_resultados.empty:
        print("\n❌ No se encontraron resultados")
        return df_resultados
    
    # Extraer nombre del alumno del archivo problemático
    df_resultados['alumno'] = df_resultados['archivo_problematico'].str.replace('.pdf', '').str.replace('_P[0-9]_.*', '', regex=True)
    
    # Ordenar por alumno y similitud (descendente)
    df_resultados = df_resultados.sort_values(['alumno', 'similitud'], ascending=[True, False])
    
    # Reordenar columnas para mejor visualización
    columnas_orden = ['alumno', 'archivo_problematico', 'similitud', 'archivo', 'tipo', 'paginas', 'ruta']
    
    df_resultados = df_resultados[columnas_orden].reset_index(drop=True)
    
    # RESUMEN FINAL
    print("\n" + "="*80)
    print("📊 RESUMEN FINAL DEL ANÁLISIS")
    print("="*80)
    
    total_archivos = len(archivos_problematicos)
    archivos_con_coincidencias = len(df_resultados[df_resultados['similitud'] > 0])
    archivos_sin_coincidencias = len(df_resultados[df_resultados['similitud'] == 0])
    
    print(f"📁 Total de archivos analizados: {total_archivos}")
    print(f"✅ Archivos con coincidencias: {archivos_con_coincidencias}")
    print(f"❌ Archivos sin coincidencias: {archivos_sin_coincidencias}")
    
    # Estadísticas de similitud
    if archivos_con_coincidencias > 0:
        coincidencias_df = df_resultados[df_resultados['similitud'] > 0]
        similitud_promedio = coincidencias_df['similitud'].mean()
        similitud_maxima = coincidencias_df['similitud'].max()
        
        print(f"📈 Similitud promedio: {similitud_promedio:.1%}")
        print(f"🎯 Similitud máxima: {similitud_maxima:.1%}")
        
        # Solo comparamos con saved, así que toda coincidencia es de tipo saved
        print(f"\n💾 Todas las coincidencias son de /saved: {archivos_con_coincidencias} archivos")
    
    return df_resultados

def comparar_pdfs_visuales_con_tqdm_solo_saved(nombre_archivo_objetivo, 
                                               ruta_saved="../data/saved/", 
                                               ruta_examenes="../data/examenes_procesados/problemático/",
                                               mostrar_detalles=False, 
                                               parar_en_100=False,
                                               umbral_similitud=0.85):
    """
    Versión modificada de comparar_pdfs_visuales_con_tqdm que SOLO compara con /saved
    Sin comparar con lotes para optimizar el análisis de problemáticos
    """
    
    # Buscar el archivo objetivo en la ruta especificada
    archivo_objetivo = None
    ruta_examenes = Path(ruta_examenes)
    
    for archivo_pdf in ruta_examenes.glob("*.pdf"):
        if nombre_archivo_objetivo.lower() in archivo_pdf.name.lower():
            archivo_objetivo = archivo_pdf
            break
    
    if not archivo_objetivo or not archivo_objetivo.exists():
        if mostrar_detalles:
            print(f"❌ No se encontró el archivo {nombre_archivo_objetivo}")
        return None
    
    if mostrar_detalles:
        print(f"🎯 Archivo encontrado: {archivo_objetivo}")
    
    # Obtener número de páginas del objetivo
    try:
        reader_objetivo = PdfReader(archivo_objetivo)
        num_paginas_objetivo = len(reader_objetivo.pages)
        if mostrar_detalles:
            print(f"📄 Páginas del archivo objetivo: {num_paginas_objetivo}")
    except Exception as e:
        if mostrar_detalles:
            print(f"❌ Error leyendo archivo objetivo: {e}")
        return None
    
    # Convertir páginas del objetivo a imágenes para comparación
    try:
        imagenes_objetivo = convert_from_path(
            archivo_objetivo, 
            dpi=100,
            fmt='jpeg'
        )
    except Exception as e:
        if mostrar_detalles:
            print(f"❌ Error convirtiendo archivo objetivo: {e}")
        return None
    
    # Lista para almacenar similitudes
    coincidencias_encontradas = []
    
    # COMPARAR SOLO CON ARCHIVOS EN /saved
    ruta_saved = Path(ruta_saved)
    
    if ruta_saved.exists():
        archivos_saved = list(ruta_saved.glob("*.pdf"))
        
        # Usar tqdm para mostrar progreso
        progress_bar = tqdm(archivos_saved, desc="💾 Comparando con /saved", leave=False, unit="archivo")
        
        for archivo_saved in progress_bar:
            try:
                reader_saved = PdfReader(archivo_saved)
                num_paginas_saved = len(reader_saved.pages)
                
                # Solo comparar si tienen el mismo número de páginas
                if num_paginas_saved == num_paginas_objetivo:
                    # Convertir a imágenes
                    imagenes_saved = convert_from_path(archivo_saved, dpi=100, fmt='jpeg')
                    
                    # Comparar cada página
                    similitud_total = comparar_imagenes_paginas(imagenes_objetivo, imagenes_saved)
                    
                    if similitud_total >= umbral_similitud:  # Usar umbral personalizado
                        similitud_info = {
                            'archivo': archivo_saved.name,
                            'ruta': str(archivo_saved),
                            'tipo': 'saved',
                            'similitud': similitud_total,
                            'paginas': num_paginas_saved
                        }
                        coincidencias_encontradas.append(similitud_info)
                        
                        if mostrar_detalles:
                            tqdm.write(f"    ✅ COINCIDENCIA: {similitud_total:.2%}")
                        
                        # PARADA AUTOMÁTICA al 100%
                        if parar_en_100 and similitud_total >= 0.999:
                            if mostrar_detalles:
                                tqdm.write(f"    🎯 ¡100% SIMILITUD ENCONTRADA! Deteniendo búsqueda...")
                            break
                        
            except Exception as e:
                if mostrar_detalles:
                    tqdm.write(f"  ❌ Error procesando {archivo_saved.name}: {e}")
                continue
        
        progress_bar.close()
    
    # Retornar coincidencias ordenadas por similitud
    if coincidencias_encontradas:
        coincidencias_encontradas.sort(key=lambda x: x['similitud'], reverse=True)
    
    return coincidencias_encontradas

In [43]:
print("🚀 Iniciando análisis RÁPIDO con parada automática...")
df_problematicos_rapido = analizar_examenes_problematicos(
    parar_en_100=True,  # ⭐ PARADA AUTOMÁTICA ACTIVADA
    umbral_similitud=0.8  # Umbral más alto para ser más selectivo
)

🚀 Iniciando análisis RÁPIDO con parada automática...
❌ No existe la carpeta 'problemático'


In [44]:
### Juntar pdfs
import os
from pathlib import Path
from pypdf import PdfReader, PdfWriter
import re

def organizar_pdfs_por_carpetas(ruta_examenes="../data/examenes_procesados/", 
                               pdfs_por_archivo=1,  # Cambiado a 1
                               grupos_objetivo=None):
    """
    Organiza los PDFs de exámenes juntándolos en grupos de N archivos por carpeta,
    ordenados alfabéticamente como aparecen en el filesystem.
    
    Args:
        ruta_examenes: Ruta base de los exámenes procesados
        pdfs_por_archivo: Número de PDFs a juntar en cada archivo resultante
        grupos_objetivo: Lista de grupos específicos a procesar (None = todos)
    """
    
    ruta_base = Path(ruta_examenes)
    
    if not ruta_base.exists():
        print(f"❌ La ruta {ruta_base} no existe")
        return
    
    # Si no se especifican grupos, procesar todos
    if grupos_objetivo is None:
        grupos_objetivo = ['IWSIM11', 'IWSIM12', 'CITIM11', 'CITIM12']
    
    print(f"🔍 Organizando PDFs - 1 PDF por archivo de práctica y grupo")
    print(f"📁 Grupos a procesar: {', '.join(grupos_objetivo)}")
    print("=" * 60)
    
    for grupo in grupos_objetivo:
        carpeta_grupo = ruta_base / grupo
        
        if not carpeta_grupo.exists():
            print(f"⚠️ La carpeta {grupo} no existe, saltando...")
            continue
            
        print(f"\n📂 Procesando grupo: {grupo}")
        
        # Buscar subcarpetas de prácticas
        subcarpetas_practica = []
        for item in carpeta_grupo.iterdir():
            if item.is_dir() and item.name.startswith("Practica_"):
                subcarpetas_practica.append(item)
        
        # Ordenar subcarpetas alfabéticamente
        subcarpetas_practica.sort(key=lambda x: x.name)
        
        for subcarpeta in subcarpetas_practica:
            print(f"  📁 Procesando {subcarpeta.name}")
            
            # Obtener todos los PDFs y ordenarlos alfabéticamente
            archivos_pdf = list(subcarpeta.glob("*.pdf"))
            # Filtrar archivos que NO sean de lotes (para evitar duplicados)
            archivos_pdf = [pdf for pdf in archivos_pdf if not re.search(r'_Lote\d+\.pdf$', pdf.name)]
            archivos_pdf.sort(key=lambda x: x.name.lower())  # Orden alfabético
            
            if not archivos_pdf:
                print(f"    ℹ️ No hay PDFs en {subcarpeta.name}")
                continue
            
            print(f"    📄 Encontrados {len(archivos_pdf)} PDFs")
            
            # Crear UN solo archivo por práctica y grupo
            practica_num = subcarpeta.name.replace("Practica_", "")
            nombre_resultado = f"{grupo}_P{practica_num}.pdf"
            ruta_resultado = subcarpeta / nombre_resultado
            
            print(f"    🔗 Creando archivo único: {nombre_resultado}")
            print(f"       📝 Archivos incluidos:")
            
            # Crear el PDF combinado
            writer = PdfWriter()
            
            for archivo in archivos_pdf:
                print(f"         • {archivo.name}")
                
                try:
                    reader = PdfReader(archivo)
                    for page in reader.pages:
                        writer.add_page(page)
                except Exception as e:
                    print(f"         ❌ Error leyendo {archivo.name}: {e}")
                    continue
            
            # Guardar el archivo combinado
            try:
                with open(ruta_resultado, 'wb') as archivo_salida:
                    writer.write(archivo_salida)
                
                total_paginas = len(writer.pages)
                print(f"       ✅ Creado: {total_paginas} páginas totales")
                
            except Exception as e:
                print(f"       ❌ Error guardando {nombre_resultado}: {e}")
    
    print(f"\n🎉 ¡Proceso completado!")

def listar_archivos_resultantes(ruta_examenes="../data/examenes_procesados/",
                               grupos_objetivo=None):
    """
    Lista los archivos creados para verificar el resultado
    """
    
    ruta_base = Path(ruta_examenes)
    
    if grupos_objetivo is None:
        grupos_objetivo = ['IWSIM11', 'IWSIM12', 'CITIM11', 'CITIM12']
    
    print("📋 ARCHIVOS CREADOS:")   
    print("=" * 50)
    
    for grupo in grupos_objetivo:
        carpeta_grupo = ruta_base / grupo
        
        if not carpeta_grupo.exists():
            continue
            
        print(f"\n📂 {grupo}:")
        
        for subcarpeta in carpeta_grupo.iterdir():
            if subcarpeta.is_dir() and subcarpeta.name.startswith("Practica_"):
                # Buscar el archivo único por práctica
                archivo_practica = list(subcarpeta.glob(f"{grupo}_P*.pdf"))
                archivo_practica = [f for f in archivo_practica if not re.search(r'_Lote\d+\.pdf$', f.name)]
                
                if archivo_practica:
                    print(f"  📁 {subcarpeta.name}:")
                    
                    for archivo in archivo_practica:
                        try:
                            reader = PdfReader(archivo)
                            num_paginas = len(reader.pages)
                            tamaño_mb = archivo.stat().st_size / (1024 * 1024)
                            
                            print(f"    📄 {archivo.name}")
                            print(f"       • Páginas: {num_paginas}")
                            print(f"       • Tamaño: {tamaño_mb:.1f} MB")
                        except Exception as e:
                            print(f"    ❌ Error leyendo {archivo.name}: {e}")


In [45]:
# Ejecutar la organización
#organizar_pdfs_por_carpetas(ruta_examenes="../data/examenes_procesados/")

# Mostrar resumen de archivos creados
print("\n" + "="*60)
#listar_archivos_resultantes()


# Check completed deriverables

In [46]:
def normalizar_texto(texto):
    """Normaliza texto eliminando acentos y caracteres especiales"""
    texto = unicodedata.normalize('NFD', texto)
    texto = ''.join(char for char in texto if unicodedata.category(char) != 'Mn')
    texto = texto.upper().strip()
    texto = re.sub(r'[^A-Z0-9\s]', '', texto)
    return texto

In [47]:
def buscar_practica_en_zips(apellidos, nombre, practica_num=3, ruta_data="./../data/"):
    """
    Busca si existe una práctica para un alumno en los archivos ZIP
    
    Args:
        apellidos: Apellidos del alumno
        nombre: Nombre del alumno
        practica_num: Número de práctica (3 o 5)
        ruta_data: Ruta base a la carpeta data
    """
    ruta_practica = Path(ruta_data) / f"Practica{practica_num}"
    
    if not ruta_practica.exists():
        return False
    
    # Normalizar apellidos y nombre
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    
    # Buscar en todos los archivos ZIP
    for archivo_zip in ruta_practica.glob("*.zip"):
        try:
            with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
                for archivo in zip_ref.namelist():
                    archivo_norm = normalizar_texto(archivo)
                    
                    # Verificar si el archivo contiene apellidos y nombre
                    if apellidos_norm in archivo_norm and nombre_norm in archivo_norm:
                        return True
        except Exception as e:
            continue
    
    return False


In [48]:
def buscar_examen_procesado(
        apellidos, 
        nombre, 
        grupo, 
        practica_num=3,
        ruta_data="./../data"
):
    """
    Busca si existe un examen procesado para un alumno
    
    Args:
        apellidos: Apellidos del alumno
        nombre: Nombre del alumno
        grupo: Grupo del alumno
        practica_num: Número de práctica (3 o 5)
    
    Returns:
        Boolean indicando si se encontró el examen
    """
    # Ruta donde se guardan los exámenes procesados
    ruta_examenes = Path(ruta_data+"/examenes_procesados") / grupo / f"Practica_{practica_num}"
    print(f"Buscando en: {ruta_examenes}")
    # Si la ruta no existe, no hay exámenes para ese grupo/práctica
    if not ruta_examenes.exists():
        return False
    
    # Normalizar apellidos y nombre
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    
    # Patrones para buscar en los nombres de archivo
    patrones = [
        # Patrón: Apellidos_Nombre
        f"{apellidos_norm}_{nombre_norm}",
        # Patrón: Apellidos Nombre
        f"{apellidos_norm} {nombre_norm}",
        # Patrón: Nombre_Apellidos
        f"{nombre_norm}_{apellidos_norm}",
        # Patrón: Nombre Apellidos
        f"{nombre_norm} {apellidos_norm}",
        # Patrón simplemente buscar ambos
        apellidos_norm
    ]
    
    # Buscar en todos los archivos de la carpeta
    for archivo in ruta_examenes.glob("*.*"):
        nombre_archivo_norm = normalizar_texto(archivo.stem)
        
        # Verificar si algún patrón coincide con el nombre del archivo
        for patron in patrones:
            if patron in nombre_archivo_norm:
                # Si además del apellido también está el nombre, es una coincidencia más fiable
                if nombre_norm in nombre_archivo_norm:
                    return True
                # Si solo coincide con el apellido y este es poco común, también se considera válido
                elif len(apellidos_norm) > 5:  # Apellidos largos son menos comunes
                    return True
    
    return False

In [49]:
def imprimir_estado_ruta(ruta: Path, descripcion: str) -> bool:
    """Imprime la existencia de una ruta con una descripción y devuelve si existe."""
    print(f"{descripcion}: {ruta.absolute()} {'✅' if ruta.exists() else '❌'}")
    return ruta.exists()

In [50]:
def verificar_practicas(ruta_base: Path, practicas: list[str]) -> dict[str, bool]:
    """Verifica una lista de carpetas de prácticas dentro de ruta_base."""
    estados = {}
    print("\nCarpetas de prácticas:")
    for nombre in practicas:
        ruta = ruta_base / nombre
        existe = imprimir_estado_ruta(ruta, f"- {nombre}")
        estados[nombre] = existe
    return estados

In [51]:
def listar_grupos_en_examenes(ruta_examenes: Path) -> list[str]:
    """Si existe la carpeta de exámenes, lista subcarpetas (grupos)."""
    grupos = []
    if ruta_examenes.exists():
        for d in ruta_examenes.iterdir():
            if d.is_dir():
                grupos.append(d.name)
        print(f"Grupos disponibles ({len(grupos)}): {', '.join(grupos) if grupos else 'ninguno'}")
    return grupos

In [52]:
def verificar_rutas(ruta_data: str = "../data") -> bool:
    """
    Función principal: verifica existencia de ruta base, 
    carpetas de prácticas y carpeta de exámenes, listando grupos.
    """
    ruta_base = Path(ruta_data)
    print(f"Verificando ruta base: {ruta_base.absolute()}")
    base_ok = imprimir_estado_ruta(ruta_base, "¿Existe ruta base?")
    
    practicas = ["Practica3", "Practica5"]
    estados_practicas = verificar_practicas(ruta_base, practicas)
    
    ruta_examenes = ruta_base / "examenes_procesados"
    print(f"\nCarpeta de exámenes:")
    examen_ok = imprimir_estado_ruta(ruta_examenes, "¿Existe carpeta de exámenes?")
    
    grupos = listar_grupos_en_examenes(ruta_examenes)
    
    return base_ok, estados_practicas, examen_ok, grupos

In [53]:
#verificar_rutas("../data")

In [54]:
def contar_examenes_por_grupo(ruta_data="../data"):
    """Cuenta cuántos exámenes hay por grupo y práctica"""
    ruta_examenes = Path(ruta_data) / "examenes_procesados"
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes.absolute()} no existe")
        return
    
    total = 0
    resultados = {}
    
    grupos = [d for d in ruta_examenes.iterdir() if d.is_dir()]
    for grupo in grupos:
        resultados[grupo.name] = {}
        
        for practica in ["Practica_3", "Practica_5"]:
            ruta_practica = grupo / practica
            if ruta_practica.exists():
                try:
                    archivos = list(ruta_practica.glob("*.*"))
                    n_archivos = len(archivos)
                    resultados[grupo.name][practica] = n_archivos
                    total += n_archivos
                except Exception as e:
                    resultados[grupo.name][practica] = f"Error: {e}"
            else:
                resultados[grupo.name][practica] = 0
    
    # Mostrar resultados
    print(f"Total exámenes encontrados: {total}")
    print("\nDesglose por grupo y práctica:")
    
    for grupo, practicas in resultados.items():
        print(f"\n{grupo}:")
        for practica, cantidad in practicas.items():
            print(f"  - {practica}: {cantidad}")
    
    return resultados

In [55]:
#contar_examenes_por_grupo("../data")

In [56]:
df_students["Nombre"] = df_students["Nombre"].str.strip().str.upper()
df_students["Apellido(s)"] = df_students["Apellido(s)"].str.strip().str.upper()
print("DataFrame original cargado:")
print(f"Total alumnos: {len(df_students)}")
#print(df_students.head())
print("\n")

DataFrame original cargado:
Total alumnos: 444




In [57]:
def probar_busqueda_examenes(apellidos, nombre, grupo, ruta_data="../data"):
    """Prueba la búsqueda de exámenes para un estudiante específico"""
    print(f"Probando búsqueda para: {apellidos}, {nombre} (Grupo: {grupo})")
    
    # Normalizar para mostrar
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    print(f"Texto normalizado: {apellidos_norm}, {nombre_norm}")
    
    # Probar prácticas 3 y 5
    for practica_num in [3, 5]:
        # Ruta donde se guardan los exámenes procesados
        ruta_examenes = Path(ruta_data) / "examenes_procesados" / grupo / f"Practica_{practica_num}"
        print(f"\nPráctica {practica_num}:")
        print(f"Ruta: {ruta_examenes}")
        
        if not ruta_examenes.exists():
            print(f"  ❌ La ruta no existe")
            continue
            
        print(f"  ✅ La ruta existe")
        
        # Listar todos los archivos en la carpeta
        print("  Archivos en la carpeta:")
        archivos = list(ruta_examenes.glob("*.*"))
        
        if not archivos:
            print("    (Carpeta vacía)")
        
        for archivo in archivos[:10]:  # Limitar a 10 para no saturar la salida
            print(f"    - {archivo.name}")
        
        if len(archivos) > 10:
            print(f"    ... y {len(archivos)-10} archivos más")
        
        # Probar búsqueda
        resultado = buscar_examen_procesado(apellidos, nombre, grupo, practica_num, ruta_data)
        print(f"\n  Resultado de búsqueda: {'✅ Encontrado' if resultado else '❌ No encontrado'}")
        
        # Si no se encontró, mostrar los patrones que se buscaron
        if not resultado:
            print("\n  Patrones buscados:")
            patrones = [
                f"{apellidos_norm}_{nombre_norm}",
                f"{apellidos_norm} {nombre_norm}",
                f"{nombre_norm}_{apellidos_norm}",
                f"{nombre_norm} {apellidos_norm}",
                apellidos_norm
            ]
            for patron in patrones:
                print(f"    - '{patron}'")

In [58]:
def listar_ejemplos_examenes(ruta_data="../data", max_por_grupo=5):
    """Lista ejemplos de nombres de archivos de exámenes por grupo"""
    ruta_examenes = Path(ruta_data) / "examenes_procesados"
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes.absolute()} no existe")
        return
    
    grupos = [d for d in ruta_examenes.iterdir() if d.is_dir()]
    print(f"Ejemplos de nombres de archivos por grupo:")
    
    for grupo in grupos:
        print(f"\n{grupo.name}:")
        
        for practica in ["Practica_3", "Practica_5"]:
            ruta_practica = grupo / practica
            if ruta_practica.exists():
                print(f"  {practica}:")
                try:
                    archivos = list(ruta_practica.glob("*.*"))[:max_por_grupo]
                    if not archivos:
                        print("    (Carpeta vacía)")
                    for archivo in archivos:
                        print(f"    - {archivo.name}")
                except Exception as e:
                    print(f"    Error al listar archivos: {e}")
            else:
                print(f"  {practica}: No existe")

In [59]:
def listar_grupos_disponibles(ruta_data="../data"):
    """Lista los nombres exactos de las carpetas de grupos disponibles"""
    ruta_examenes = Path(ruta_data) / "examenes_procesados"
    
    if not ruta_examenes.exists():
        print(f"La ruta {ruta_examenes.absolute()} no existe")
        return []
    
    grupos = [d.name for d in ruta_examenes.iterdir() if d.is_dir()]
    print(f"Grupos disponibles ({len(grupos)}): {', '.join(grupos)}")
    
    return grupos

### Buscar un alumno concreto

In [66]:
def lookFor(df, lastnames, firstname, verbose=0):
    # Clean search inputs
    clean_lastnames = limpiar_texto(lastnames)
    clean_firstname = limpiar_texto(firstname)
    
    # Create a temporary DataFrame with cleaned columns for comparison
    temp_df = df.copy()
    temp_df['Clean_Apellidos'] = df['Apellido(s)'].apply(limpiar_texto)
    temp_df['Clean_Nombre'] = df['Nombre'].apply(limpiar_texto)
    
    # Handle different search combinations
    if clean_lastnames and clean_firstname:
        # Search by both lastname and firstname
        result = temp_df[(temp_df['Clean_Apellidos'] == clean_lastnames) & 
                        (temp_df['Clean_Nombre'] == clean_firstname)]
    elif clean_lastnames:
        # Search only by lastname
        result = temp_df[temp_df['Clean_Apellidos'] == clean_lastnames]
    elif clean_firstname:
        # Search only by firstname
        result = temp_df[temp_df['Clean_Nombre'] == clean_firstname]
    else:
        # Neither name provided, return empty DataFrame
        result = df.head(0)
    
    # Using original df for display but with the filtered indexes
    result_orig = df.loc[result.index]
    
    if not result_orig.empty:
        if verbose > 0:
            if len(result_orig) == 1:
                grupo = result_orig['Grupos'].iloc[0]
                print(f"El grupo de {firstname if firstname else ''} {lastnames if lastnames else ''} es: {grupo}")
            else:
                print(f"Se encontraron {len(result_orig)} coincidencias:")
            print("\nInformación completa:")
            print(result_orig[['Nombre', 'Apellido(s)', 'Grupos']].to_string(index=False))
    else:
        print(f"No se encontró a {firstname if firstname else ''} {lastnames if lastnames else ''} en el DataFrame")
    
    return result_orig

In [68]:
apellidos = 'Rodríguez Fernández'
nombre = 'Víctor'
lookFor(df_students, apellidos, nombre, verbose=1)

Se encontraron 2 coincidencias:

Información completa:
Nombre         Apellido(s)              Grupos
VICTOR RODRIGUEZ FERNANDEZ          Profesores
VICTOR RODRIGUEZ FERNANDEZ CITIM12, Profesores


,Nombre,Apellido(s),Dirección de correo,Grupos
192,VICTOR,RODRIGUEZ FERNANDEZ,victor.rfernandez@upm.es,Profesores
393,VICTOR,RODRIGUEZ FERNANDEZ,victor.rfernandez@upm.es,"CITIM12, Profesores"


In [71]:
def mapear_grupos_a_carpetas(ruta_data="../data"):
    """
    Crea un mapeo de códigos de grupo a nombres de carpetas reales
    
    Args:
        ruta_data: Ruta base a la carpeta data
    
    Returns:
        Diccionario que mapea códigos de grupo a nombres de carpeta
    """
    ruta_examenes = Path(ruta_data) / "examenes_procesados"
    
    if not ruta_examenes.exists():
        return {}
    
    # Obtener las carpetas de grupos disponibles
    carpetas_grupos = [d.name for d in ruta_examenes.iterdir() if d.is_dir()]
    
    # Crear un mapeo de códigos de grupos a carpetas
    mapeo = {}
    for carpeta in carpetas_grupos:
        mapeo[carpeta] = carpeta  # Mapeo directo para coincidencias exactas
        
        # También mapear códigos que estén contenidos en nombres de carpeta
        for codigo in ['IWSIM11', 'IWSIM12', 'CITIM11', 'CITIM12', 'CITIT11', 'IWSIT12']:
            if codigo in carpeta and codigo not in mapeo:
                mapeo[codigo] = carpeta
    
    return mapeo

In [72]:
def buscar_examen_mejorado(apellidos, nombre, grupo, practica_num, ruta_data="../data", verbose=False):
    """
    Función mejorada para buscar exámenes con patrones más flexibles
    
    Args:
        apellidos: Apellidos del alumno
        nombre: Nombre del alumno
        grupo: Grupo del alumno (nombre de carpeta)
        practica_num: Número de práctica (3 o 5)
        ruta_data: Ruta base a la carpeta data
        verbose: Si es True, muestra información detallada de la búsqueda
    
    Returns:
        Boolean indicando si se encontró el examen
    """
    # Ruta donde se guardan los exámenes procesados
    ruta_examenes = Path(ruta_data) / "examenes_procesados" / grupo / f"Practica_{practica_num}"
    
    if verbose:
        print(f"Buscando en: {ruta_examenes}")
    
    # Si la ruta no existe, no hay exámenes para ese grupo/práctica
    if not ruta_examenes.exists():
        if verbose:
            print(f"La ruta {ruta_examenes} no existe")
        return False
    
    # Normalizar textos
    apellidos_norm = normalizar_texto(apellidos)
    nombre_norm = normalizar_texto(nombre)
    
    # Separar apellidos si hay varios
    apellidos_lista = apellidos_norm.split()
    
    # Patrones para buscar en los nombres de archivo
    patrones = [
        f"{apellidos_norm}_{nombre_norm}",         # Apellidos_Nombre
        f"{apellidos_norm} {nombre_norm}",         # Apellidos Nombre
        f"{nombre_norm}_{apellidos_norm}",         # Nombre_Apellidos
        f"{nombre_norm} {apellidos_norm}",         # Nombre Apellidos
        apellidos_norm,                            # Solo apellidos
    ]
    
    # Añadir patrones específicos para el formato P5_GRUPO
    patrones_practica = [
        f"{apellidos_norm}_{nombre_norm}_P{practica_num}_{grupo}",
        f"{apellidos_norm}_{nombre_norm}_P{practica_num}",
        f"{nombre_norm}_{apellidos_norm}_P{practica_num}"
    ]
    
    todos_patrones = patrones + patrones_practica
    
    # Buscar en todos los archivos de la carpeta
    for archivo in ruta_examenes.glob("*.*"):
        nombre_archivo_norm = normalizar_texto(archivo.stem)
        
        # Verificar si algún patrón coincide con el nombre del archivo
        for patron in todos_patrones:
            if patron in nombre_archivo_norm:
                if verbose:
                    print(f"✅ Encontrado con patrón '{patron}': {archivo.name}")
                return True
                
        # Si no hay coincidencia exacta con los patrones, probar si están presentes 
        # tanto el nombre como al menos un apellido (para el formato libre)
        if nombre_norm in nombre_archivo_norm:
            for apellido in apellidos_lista:
                if len(apellido) > 3 and apellido in nombre_archivo_norm:  # Apellido con longitud razonable
                    if verbose:
                        print(f"✅ Encontrado parcial: {archivo.name}")
                    return True
    
    if verbose:
        print(f"❌ No se encontró ninguna coincidencia para {apellidos_norm}, {nombre_norm}")
    return False

In [75]:
def verificar_todas_las_practicas(df, ruta_data="../data", grupo_filtro=None, verbose=False):
    """
    Añade columnas de verificación de prácticas y exámenes al DataFrame existente
    
    Args:
        df: DataFrame con columnas 'Nombre' y 'Apellido(s)'
        ruta_data: Ruta base a la carpeta data
        grupo_filtro: Opcional, nombre del grupo para filtrar (ej. "IWSIM11")
        verbose: Si es True, muestra información detallada
    
    Returns:
        DataFrame modificado con las nuevas columnas
    """
    # Obtener el mapeo de grupos a carpetas
    mapeo_grupos = mapear_grupos_a_carpetas(ruta_data)
    
    if verbose:
        print(f"Mapeo de grupos a carpetas: {mapeo_grupos}")
    
    # Filtrar el DataFrame por grupo si se especificó
    if grupo_filtro:
        if verbose:
            print(f"Filtrando por grupo: {grupo_filtro}")
        df = df[df['Grupos'].str.contains(grupo_filtro, case=False, na=False)]
        print(f"Encontrados {len(df)} estudiantes del grupo {grupo_filtro}")
    
    # Crear copias para evitar warnings
    df_resultado = df.copy()
    
    # Inicializar las nuevas columnas para prácticas
    df_resultado['Presentada_3'] = 0
    df_resultado['Comentario_3'] = 'NP'
    df_resultado['Presentada_5'] = 0
    df_resultado['Comentario_5'] = 'NP'
    
    # Inicializar las nuevas columnas para exámenes
    df_resultado['Examen Practica 3'] = None
    df_resultado['Examen Practica 5'] = None
    df_resultado['Examen Practica 3 entregado'] = 0
    df_resultado['Examen Practica 5 entregado'] = 0
    
    print("Verificando entregas de prácticas y exámenes...")
    print("="*50)
    
    # Contadores para estadísticas
    practicas_3_encontradas = 0
    practicas_5_encontradas = 0
    examenes_3_encontrados = 0
    examenes_5_encontrados = 0
    total_alumnos = len(df_resultado)
    
    # Para cada alumno en el DataFrame
    for idx, row in df_resultado.iterrows():
        nombre = str(row['Nombre'])
        apellidos = str(row['Apellido(s)'])
        grupo_codigo = str(row.get('Grupos', 'extraviado'))
        
        # Obtener la carpeta real del grupo (si existe)
        grupo_carpeta = mapeo_grupos.get(grupo_codigo, grupo_codigo)
        
        # Verificar Práctica 3
        tiene_practica3 = buscar_practica_en_zips(apellidos, nombre, 3, ruta_data)
        if tiene_practica3:
            df_resultado.loc[idx, 'Presentada_3'] = 1
            df_resultado.loc[idx, 'Comentario_3'] = ''
            practicas_3_encontradas += 1
        
        # Verificar Práctica 5
        tiene_practica5 = buscar_practica_en_zips(apellidos, nombre, 5, ruta_data)
        if tiene_practica5:
            df_resultado.loc[idx, 'Presentada_5'] = 1
            df_resultado.loc[idx, 'Comentario_5'] = ''
            practicas_5_encontradas += 1
        
        # Verificar Examen de Práctica 3
        tiene_examen3 = buscar_examen_mejorado(
            apellidos, nombre, grupo_carpeta, 3, ruta_data, verbose=False
        )
        if tiene_examen3:
            df_resultado.loc[idx, 'Examen Practica 3 entregado'] = 1
            examenes_3_encontrados += 1
            # Si no tiene práctica pero sí examen, poner 0
            if not tiene_practica3:
                df_resultado.loc[idx, 'Examen Practica 3'] = 0
        else:
            # No ha presentado el examen
            if df_resultado.loc[idx, 'Comentario_3']:
                df_resultado.loc[idx, 'Comentario_3'] += '. Examen no presentado'
            else:
                df_resultado.loc[idx, 'Comentario_3'] = 'Examen no presentado'
        
        # Verificar Examen de Práctica 5
        tiene_examen5 = buscar_examen_mejorado(
            apellidos, nombre, grupo_carpeta, 5, ruta_data, verbose=False
        )
        if tiene_examen5:
            df_resultado.loc[idx, 'Examen Practica 5 entregado'] = 1
            examenes_5_encontrados += 1
            # Si no tiene práctica pero sí examen, poner 0
            if not tiene_practica5:
                df_resultado.loc[idx, 'Examen Practica 5'] = 0
        else:
            # No ha presentado el examen
            if df_resultado.loc[idx, 'Comentario_5']:
                df_resultado.loc[idx, 'Comentario_5'] += '. Examen no presentado'
            else:
                df_resultado.loc[idx, 'Comentario_5'] = 'Examen no presentado'
        
        # Mostrar progreso
        status_3_practica = "✓" if tiene_practica3 else "✗"
        status_5_practica = "✓" if tiene_practica5 else "✗"
        status_3_examen = "📝" if tiene_examen3 else "❌"
        status_5_examen = "📝" if tiene_examen5 else "❌"
        
        if verbose: print(f"{status_3_practica}P3 {status_3_examen}E3 | {status_5_practica}P5 {status_5_examen}E5 | {apellidos}, {nombre} ({grupo_carpeta})")
    
    print("="*50)
    print(f"RESUMEN:")
    print(f"Total alumnos verificados: {total_alumnos}")
    print(f"Práctica 3 - Entregadas: {practicas_3_encontradas} | No entregadas: {total_alumnos - practicas_3_encontradas}")
    print(f"Práctica 5 - Entregadas: {practicas_5_encontradas} | No entregadas: {total_alumnos - practicas_5_encontradas}")
    print(f"Examen 3 - Presentados: {examenes_3_encontrados} | No presentados: {total_alumnos - examenes_3_encontrados}")
    print(f"Examen 5 - Presentados: {examenes_5_encontrados} | No presentados: {total_alumnos - examenes_5_encontrados}")
    
    return df_resultado

In [76]:
df_students_info_full = verificar_todas_las_practicas(
    df_students,
    ruta_data="../data",
    #grupo_filtro="IWSIM11",
    verbose=False
)

Verificando entregas de prácticas y exámenes...
RESUMEN:
Total alumnos verificados: 444
Práctica 3 - Entregadas: 92 | No entregadas: 352
Práctica 5 - Entregadas: 302 | No entregadas: 142
Examen 3 - Presentados: 50 | No presentados: 394
Examen 5 - Presentados: 143 | No presentados: 301


In [ ]:
#df_students_info_full.to_excel(
#    "../data/df_students_info_full.xlsx",
#    index=False
#)

# Grades OCR + json_students

In [78]:
prompt = """
    Extract the grade of this exam: 
    - It is a number between 0 and 10 
    - at most two decimals, may be written with ',' or '.'), 
    - It is written with red at the top of the image
    - It is normally preceded by 'Nota final'
    If you see multiple numbers in the image:
    - take the biggest one as the final grade, at one corner of the paper   
    The field of your JSON output will have the name "grade".
"""

In [80]:
try: 
    pdf_openai_parser_safeGroup(
        pdf_path    = "../data/examenes_corregidos/CITIM11/Practica_3/ARIAS_CASADO_ALBA_P3_CITIM11.pdf",
        model       = model,
        prompt      = prompt.format(json_info=json_students)
    )
except Exception as e:
    print(f"Error al procesar el PDF: {e}")
    grade = None

Image 1 missing required fields, trying next image...
Image 2 missing required fields, trying next image...
No valid data found in any image
Error al procesar el PDF: 'NoneType' object has no attribute 'get'


In [84]:
import os
import io
import json
import base64
import re
import requests
from pathlib import Path
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown, FloatText, HTML
from IPython.display import display
import pandas as pd

In [85]:
def extract_grade_from_pdf(
    pdf_path: str,
    model: str,
    prompt: str,
    api_key: str,
    pages: tuple = (1, 2),
    dpi: int = 150
) -> float | None:
    """
    Convierte las páginas indicadas de pdf_path a JPEGs y las envía
    al modelo para extraer la nota. Devuelve la nota (float) o None.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    for p in pages:
        imgs = convert_from_path(pdf_path, first_page=p, last_page=p, dpi=dpi, fmt='jpeg')
        if not imgs:
            continue
        buf = io.BytesIO()
        imgs[0].save(buf, 'JPEG')
        img_b64 = base64.b64encode(buf.getvalue()).decode()
        payload = {
            "model": model,
            "response_format": {"type": "json_object"},
            "messages": [
                {"role": "system", "content": "Extract grade."},
                {"role": "user", "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
                ]}
            ],
            "max_tokens": 150
        }
        r = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        if r.status_code != 200:
            continue
        data = r.json()["choices"][0]["message"]["content"]
        try:
            grade = float(str(json.loads(data).get("grade")).replace(",", "."))
            return grade
        except:
            continue
    return None


In [90]:
extract_grade_from_pdf(
    model   = model, 
    prompt  = prompt, 
    api_key = api_key, 
    pdf_path = "../example_data/1.pdf"
)

8.75

In [ ]:
def obtenerNotasTodosAlumnos(df, base_path):
    """
    Extrae las notas de todos los alumnos que se han presentado a examen
    usando OCR con OpenAI.
    
    Args:
        df: DataFrame con información de estudiantes
        base_path: Ruta base a la carpeta de exámenes corregidos
    
    Returns:
        DataFrame actualizado con las notas extraídas
    """
    
    df_resultado = df.copy()
    base_path = Path(base_path)
    
    # Contadores para estadísticas
    procesados = 0
    errores = 0
    no_encontrados = 0
    
    print("🔍 Iniciando extracción de notas...")
    print("=" * 50)
    
    for idx, row in df_resultado.iterrows():
        apellidos = str(row['Apellido(s)']).replace(' ', '_').upper()
        nombre = str(row['Nombre']).replace(' ', '_').upper()
        grupo = str(row['Grupos'])
        
        # Procesar prácticas 3 y 5
        for practica_num in [3, 5]:
            print(f"{grupo} 🔎 Procesando: {apellidos}, {nombre} | Practica {practica_num}")
            col_entregado = f'Examen Practica {practica_num} entregado'
            col_nota = f'Examen Practica {practica_num}'
            
            # Solo procesar si se ha presentado al examen
            if row.get(col_entregado, 0) == 1:
                # Construir ruta esperada del archivo
                ruta_practica = base_path / grupo / f"Practica_{practica_num}"
                
                if not ruta_practica.exists():
                    print(f"⚠️ No existe la carpeta: {ruta_practica}")
                    continue
                
                # Buscar archivo que coincida con el patrón del alumno
                archivo_encontrado = None
                
                # Buscar archivos PDF que contengan el patrón
                for archivo_pdf in ruta_practica.glob("*.pdf"):
                    nombre_archivo = archivo_pdf.stem.upper()
                    
                    # Verificar si el archivo contiene apellidos y nombre
                    if  (apellidos in nombre_archivo and nombre in nombre_archivo): 
                        archivo_encontrado = archivo_pdf
                        break
                
                if archivo_encontrado:
                    try:
                        # Extraer nota usando OCR
                        nota = extract_grade_from_pdf(
                            pdf_path=str(archivo_encontrado),
                            model=model,
                            prompt=prompt,
                            api_key=api_key
                        )
                        
                        if nota is not None:
                            df_resultado.loc[idx, col_nota] = nota
                            print(f"✅ [ {grupo} ] {apellidos}, {nombre} - P{practica_num}: {nota:.2f}")
                            procesados += 1
                        else:
                            print(f"[ {grupo} ] ❌ [ {grupo} ] {apellidos}, {nombre} - P{practica_num}: No se pudo extraer nota")
                            errores += 1
                            
                    except Exception as e:
                        print(f"[ {grupo} ] ❌ Error procesando {archivo_encontrado.name}: {e}")
                        errores += 1
                else:
                    print(f"📁 [ {grupo} ] {apellidos}, {nombre} - P{practica_num}: Archivo no encontrado")
                    no_encontrados += 1
    
    print("=" * 50)
    print(f"📊 RESUMEN:")
    print(f"✅ Notas extraídas exitosamente: {procesados}")
    print(f"❌ Errores en extracción: {errores}")
    print(f"📁 Archivos no encontrados: {no_encontrados}")
    
    return df_resultado

In [96]:
obtenerNotasTodosAlumnos(df_students_info_full, "../data/examenes_corregidos/")

🔍 Iniciando extracción de notas...
📁 [ IWSIM12 ] AGUIRRE_HERVIAS, JAVIER - P5: Archivo no encontrado
✅ [ IWSIM12 ] ALVAREZ_AREVALO, MIGUEL - P3: 10.00
✅ [ IWSIM12 ] AUSIN_MORENO, MARCOS - P5: 7.00
✅ [ IWSIM12 ] AYALA_MAYA, JULIO - P5: 10.00
✅ [ IWSIM11 ] BEAUTELL_NAVARRO, HUGO - P3: 8.25
📁 [ IWSIM11 ] BEAUTELL_NAVARRO, HUGO - P5: Archivo no encontrado
✅ [ IWSIM11 ] BLANCO_MARCHAL, SIMON - P5: 10.00
✅ [ IWSIM11 ] BRAVO_CUEVA, ALVARO - P5: 0.00
✅ [ IWSIM11 ] CARRASCO_PARDO, SERGIO - P3: 6.50
✅ [ IWSIM11 ] CARRASCO_PARDO, SERGIO - P5: 2.00
✅ [ IWSIM12 ] DE_LOS_MOZOS_DE_LA_CRUZ, DIEGO - P5: 9.00
✅ [ IWSIM12 ] DOMINGUEZ_ALVAREZ, JAVIER - P5: 2.00


KeyboardInterrupt: 

In [92]:
def run_ocr_batch(df, row_to_pdf_path):
    df = df.copy()
    notas = []
    for idx, row in df.iterrows():
        pdf_path = row_to_pdf_path(row)  # función tuya que mapea fila→ruta pdf
        nota = extract_grade_from_pdf(pdf_path, model, prompt, api_key)
        notas.append(nota if nota is not None else float('nan'))
    df["Nota_OCR"] = notas
    return df

In [ ]:
df_con_notas = run_ocr_batch(
    df_students_info_full,
    lambda row: f"../data/examenes_corregidos/{row['Grupos']}/Practica_{row['Práctica']}/{row['Archivo']}"
)
display(df_con_notas.head())

In [ ]:
class ExamGradeReviewer:
    def __init__(
        self,
        base_dir: str = "../data/examenes_corregidos/",
        model: str = "gpt-4o",
        df_students: pd.DataFrame = None,
        prompt: str = None
    ):
        # Directorios y estado inicial
        self.base_dir = Path(base_dir)
        self.last_dir = Path("./data/last"); self.last_dir.mkdir(exist_ok=True, parents=True)
        self.last_state_path = self.last_dir / "last_visit.json"

        # Config OpenAI
        self.model = model
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY no está configurada")
        self.prompt = prompt or (
            "Extract the grade of this exam:\n"
            "- It is a number between 0 and 10\n"
            "- At most two decimals (',' or '.')\n"
            "- Written in red at the top, normally preceded by 'Nota final'\n"
            "Output JSON with field 'grade'."
        )
        self.headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.api_key}"}

        # Data students
        self.df_students = df_students.copy() if df_students is not None else pd.DataFrame()

        # Dropdown widgets
        self.dropdown_group = Dropdown(description='Grupo:', layout=Layout(width='200px'))
        self.dropdown_practice = Dropdown(description='Práctica:', layout=Layout(width='200px'))
        self.dropdown_exam = Dropdown(description='Examen:', layout=Layout(width='350px'))
        self.dropdown_last = Dropdown(description='Apellido:', layout=Layout(width='200px'))
        self.dropdown_first = Dropdown(description='Nombre:', layout=Layout(width='200px'))

        # Grade widgets
        self.grade_label = Label('Nota:', layout=Layout(width='50px'))
        self.grade_value = HTML(layout=Layout(width='80px'))
        self.grade_input = FloatText(0.0, min=0, max=10, step=0.1, layout=Layout(width='80px'))
        self.btn_save_grade = Button(description='Guardar nota', button_style='success', layout=Layout(width='120px'))
        self.btn_extract = Button(description='Extraer nota', button_style='info', layout=Layout(width='120px'))
        self.btn_export = Button(description='Exportar Excel', button_style='primary', layout=Layout(width='150px'))
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='100px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='100px'))

        # Output areas
        self.image_out = Output(layout=Layout(width='60%', overflow='hidden'))
        self.table_out = Output(layout=Layout(width='40%', overflow='auto'))
        self.status = Label()
        self.message = HTML()

        # Bind events
        self.dropdown_group.observe(self._on_group_change, names='value')
        self.dropdown_practice.observe(self._on_practice_change, names='value')
        self.dropdown_exam.observe(self._on_exam_change, names='value')
        self.dropdown_last.observe(lambda _: self._update_first_options(), names='value')
        self.btn_prev.on_click(lambda b: self._navigate(-1))
        self.btn_next.on_click(lambda b: self._navigate(1))
        self.btn_extract.on_click(lambda b: self._extract_grade())
        self.btn_save_grade.on_click(self._on_save_grade)
        self.btn_export.on_click(self._on_export)

        # Build interface and initialize
        self._build_interface()
        self._find_groups_and_exams()
        self._load_last()

    def _build_interface(self):
        controls = VBox([
            HBox([self.dropdown_group, self.dropdown_practice, self.dropdown_exam]),
            HBox([self.dropdown_last, self.dropdown_first]),
            HBox([self.grade_label, self.grade_value, self.grade_input,
                  self.btn_save_grade, self.btn_extract, self.btn_export]),
            HBox([self.btn_prev, self.btn_next]),
            HBox([self.message]), HBox([self.status])
        ])
        content = HBox([self.image_out, self.table_out], layout=Layout(width='100%'))
        display(VBox([controls, content]))

    def _find_groups_and_exams(self):
        self.status.value = 'Buscando grupos y exámenes...'
        groups = [d.name for d in self.base_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]
        self.dropdown_group.options = sorted(groups)

    def _on_group_change(self, change):
        grp = change['new']
        if not grp: return
        prac_dir = self.base_dir / grp
        self.dropdown_practice.options = sorted([d.name for d in prac_dir.iterdir() if d.is_dir() and d.name.startswith('Practica_')])
        self._update_student_dropdowns()

    def _on_practice_change(self, change):
        prac = change['new']
        if not prac: return
        ex_dir = self.base_dir / self.dropdown_group.value / prac
        self.dropdown_exam.options = sorted([p.name for p in ex_dir.glob('*.pdf')])

    def _on_exam_change(self, change):
        ex = change['new']
        if not ex: return
        # show PDF, extract grade, load last
        self.current_pdf = self.base_dir / self.dropdown_group.value / self.dropdown_practice.value / ex
        self._show_exam_image()
        self._extract_grade()
        self._save_last()
        # Auto-select student AFTER loading the new exam
        self._auto_select_student()

    def _update_student_dropdowns(self):
        # filter df by group
        if 'Grupos' in self.df_students.columns:
            grp = self.dropdown_group.value
            df_grp = self.df_students[self.df_students['Grupos'] == grp]
        else:
            df_grp = self.df_students
        last_opts = sorted(df_grp['Apellido(s)'].dropna().unique())
        self.dropdown_last.options = last_opts
        if last_opts:
            self.dropdown_last.value = last_opts[0]
            self._update_first_options()
        else:
            self.dropdown_last.value = None
            self.dropdown_first.options = []

    def _update_first_options(self):
        last = self.dropdown_last.value
        if last and not self.df_students.empty:
            names = sorted(self.df_students[self.df_students['Apellido(s)'] == last]['Nombre'].unique())
            self.dropdown_first.options = names
            if names:
                self.dropdown_first.value = names[0]
        else:
            self.dropdown_first.options = []
            
    def _auto_select_student(self):
        """Selecciona automáticamente el estudiante basado en el nombre del archivo"""
        # Obtener nombre del archivo actual
        filename = self.dropdown_exam.value
        if not filename:
            return
        
        # Función para normalizar texto (quitar acentos y mayúsculas)
        def normalize(text):
            import unicodedata
            if not text: return ""
            text = unicodedata.normalize('NFD', text.upper())
            return ''.join(c for c in text if not unicodedata.combining(c))
        
        # Extraer primera palabra (apellido)
        first_word = filename.split('_')[0] if '_' in filename else filename.split('.')[0]
        first_word_norm = normalize(first_word)
        
        # 1. PRIORIDAD ABSOLUTA: Buscar apellido que comience exactamente con la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            
            # Normalizar el apellido del dropdown
            last_norm = normalize(last)
            last_parts = last_norm.split()
            
            # Si el apellido comienza exactamente con la primera palabra del archivo
            if last_parts and last_parts[0] == first_word_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 2. SEGUNDA PRIORIDAD: Buscar apellido que contenga la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            last_norm = normalize(last)
            
            # Si el apellido contiene la primera palabra completa
            if first_word_norm in last_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 3. ÚLTIMA OPCIÓN: Búsqueda por coincidencia parcial si las anteriores fallan
        # Buscar por tokens en todo el nombre del archivo
        filename_parts = filename.replace('.pdf', '').split('_')
        
        for part in filename_parts:
            part_norm = normalize(part)
            if len(part_norm) < 3:  # Ignorar partes muy cortas
                continue
            
            for last in self.dropdown_last.options:
                if not last: continue
                last_norm = normalize(last)
                
                if part_norm in last_norm:
                    self.dropdown_last.value = last
                    self._update_first_options()
                    return

    def _navigate(self, step):
        opts = list(self.dropdown_exam.options)
        if not opts: return
        idx = opts.index(self.dropdown_exam.value) + step
        if 0 <= idx < len(opts):
            self.dropdown_exam.value = opts[idx]
        else:
            self.message.value = '<span style="color:orange;">No hay más exámenes</span>'

    def _show_exam_image(self):
        self.image_out.clear_output()
        with self.image_out:
            imgs = convert_from_path(self.current_pdf, first_page=1, last_page=2, dpi=150, fmt='jpeg')
            if not imgs:
                print('No hay imágenes')
            else:
                cropped = [im.crop((0, 0, im.width, 225)) for im in imgs]
                fig, axes = plt.subplots(nrows=len(cropped), figsize=(8, 2 * len(cropped)), tight_layout=True)
                if len(cropped) == 1:
                    axes = [axes]
                for ax, im in zip(axes, cropped):
                    ax.imshow(im)
                    ax.axis('off')
                plt.show()
        self._update_student_table()

    def _extract_grade(self):
        if not hasattr(self, 'current_pdf') or not self.current_pdf:
            return
        self.status.value = 'Extrayendo nota...'
        for p in (1, 2):
            imgs = convert_from_path(self.current_pdf, first_page=p, last_page=p, dpi=150, fmt='jpeg')
            if not imgs:
                continue
            buf = io.BytesIO()
            imgs[0].save(buf, 'JPEG')
            img_b64 = base64.b64encode(buf.getvalue()).decode()
            payload = {
                'model': self.model,
                'response_format': {'type': 'json_object'},
                'messages': [
                    {'role': 'system', 'content': 'Extract grade.'},
                    {'role': 'user', 'content': [
                        {'type': 'text', 'text': self.prompt},
                        {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{img_b64}'}}
                    ]}
                ],
                'max_tokens': 150
            }
            r = requests.post('https://api.openai.com/v1/chat/completions', headers=self.headers, json=payload)
            if r.status_code != 200:
                continue
            data = json.loads(r.json()['choices'][0]['message']['content'])
            try:
                grd = float(str(data.get('grade')).replace(',', '.'))
            except:
                continue
            self.current_grade = grd; self.current_grade_type = 'auto'
            self.grade_value.value = f"<span style='color:#000088;font-weight:bold;'>{grd:.2f}</span>"
            self.grade_input.value = grd
            self.status.value = f'Nota extraída pág {p}'
            break

    def _on_save_grade(self, b):
        g = self.grade_input.value
        if g < 0 or g > 10:
            self.message.value = '<span style="color:red;">Nota 0-10</span>'
            return
        # Update grade widgets
        self.current_grade = g; self.current_grade_type = 'manual'
        self.grade_value.value = f"<span style='color:#008800;font-weight:bold;'>{g:.2f}</span>"

        # Determine column
        m = re.search(r"Practica[_ ]?(\d+)", self.dropdown_practice.value or "")
        col = f"Examen Practica {m.group(1)}" if m else None
        last = self.dropdown_last.value; first = self.dropdown_first.value
        if col and col in self.df_students.columns:
            mask = (self.df_students['Apellido(s)'] == last) & (self.df_students['Nombre'] == first)
            if mask.any():
                self.df_students.loc[mask, col] = g
        self.message.value = f'<span style="color:green;">Nota {g:.2f} guardada</span>'
        self._update_student_table()

    def _update_student_table(self):
        if self.df_students.empty:
            return
        self.table_out.clear_output()
        with self.table_out:
            df = self.df_students
            cols = ['Nombre', 'Apellido(s)'] + [c for c in df.columns if c.startswith('Examen Practica')] + (['grade'] if 'grade' in df.columns else [])
            last = self.dropdown_last.value; first = self.dropdown_first.value
            mask = (df['Apellido(s)'] == last) & (df['Nombre'] == first)
            if mask.any():
                idx_list = list(df.index)
                pos = idx_list.index(df[mask].index[0])
                start = max(0, pos - 2)
                end = min(len(idx_list), pos + 3)
                display(df.loc[idx_list[start:end], cols])
            else:
                display(df[cols].head(5))

    def _on_export(self, b):
        out_file = self.last_dir / 'students.xlsx'
        self.df_students.to_excel(out_file, index=False)
        self.message.value = f'<span style="color:green;">Exportado a {out_file}</span>'

    def _save_last(self):
        state = {
            'group': self.dropdown_group.value,
            'practice': self.dropdown_practice.value,
            'exam': self.dropdown_exam.value,
            'last': self.dropdown_last.value,
            'first': self.dropdown_first.value
        }
        self.last_state_path.write_text(json.dumps(state))

    def _load_last(self):
        if not self.last_state_path.exists():
            return
        try:
            data = json.loads(self.last_state_path.read_text())
            if data.get('group') in self.dropdown_group.options:
                self.dropdown_group.value = data['group']
            if data.get('practice') in self.dropdown_practice.options:
                self.dropdown_practice.value = data['practice']
            if data.get('exam') in self.dropdown_exam.options:
                self.dropdown_exam.value = data['exam']
            if data.get('last') in self.dropdown_last.options:
                self.dropdown_last.value = data['last']
                self._update_first_options()
            if data.get('first') in self.dropdown_first.options:
                self.dropdown_first.value = data['first']
        except:
            pass

SyntaxError: invalid character '…' (U+2026) (3503184244.py, line 3)

In [ ]:


class ExamGradeReviewer:
    def __init__(
        self,
        base_dir: str = "../data/examenes_corregidos/",
        model: str = "gpt-4o",
        df_students: pd.DataFrame = None,
        prompt: str = None
    ):
        # Directorios y estado inicial
        self.base_dir = Path(base_dir)
        self.last_dir = Path("./data/last"); self.last_dir.mkdir(exist_ok=True, parents=True)
        self.last_state_path = self.last_dir / "last_visit.json"

        # Config OpenAI
        self.model = model
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY no está configurada")
        self.prompt = prompt or (
            "Extract the grade of this exam:\n"
            "- It is a number between 0 and 10\n"
            "- At most two decimals (',' or '.')\n"
            "- Written in red at the top, normally preceded by 'Nota final'\n"
            "Output JSON with field 'grade'."
        )
        self.headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.api_key}"}

        # Data students
        self.df_students = df_students.copy() if df_students is not None else pd.DataFrame()

        # Dropdown widgets
        self.dropdown_group = Dropdown(description='Grupo:', layout=Layout(width='200px'))
        self.dropdown_practice = Dropdown(description='Práctica:', layout=Layout(width='200px'))
        self.dropdown_exam = Dropdown(description='Examen:', layout=Layout(width='350px'))
        self.dropdown_last = Dropdown(description='Apellido:', layout=Layout(width='200px'))
        self.dropdown_first = Dropdown(description='Nombre:', layout=Layout(width='200px'))

        # Grade widgets
        self.grade_label = Label('Nota:', layout=Layout(width='50px'))
        self.grade_value = HTML(layout=Layout(width='80px'))
        self.grade_input = FloatText(0.0, min=0, max=10, step=0.1, layout=Layout(width='80px'))
        self.btn_save_grade = Button(description='Guardar nota', button_style='success', layout=Layout(width='120px'))
        self.btn_extract = Button(description='Extraer nota', button_style='info', layout=Layout(width='120px'))
        self.btn_export = Button(description='Exportar Excel', button_style='primary', layout=Layout(width='150px'))
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='100px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='100px'))

        # Output areas
        self.image_out = Output(layout=Layout(width='60%', overflow='hidden'))
        self.table_out = Output(layout=Layout(width='40%', overflow='auto'))
        self.status = Label()
        self.message = HTML()

        # Bind events
        self.dropdown_group.observe(self._on_group_change, names='value')
        self.dropdown_practice.observe(self._on_practice_change, names='value')
        self.dropdown_exam.observe(self._on_exam_change, names='value')
        self.dropdown_last.observe(lambda _: self._update_first_options(), names='value')
        self.btn_prev.on_click(lambda b: self._navigate(-1))
        self.btn_next.on_click(lambda b: self._navigate(1))
        self.btn_extract.on_click(lambda b: self._extract_grade())
        self.btn_save_grade.on_click(self._on_save_grade)
        self.btn_export.on_click(self._on_export)

        # Build interface and initialize
        self._build_interface()
        self._find_groups_and_exams()
        self._load_last()

    def _build_interface(self):
        controls = VBox([
            HBox([self.dropdown_group, self.dropdown_practice, self.dropdown_exam]),
            HBox([self.dropdown_last, self.dropdown_first]),
            HBox([self.grade_label, self.grade_value, self.grade_input,
                  self.btn_save_grade, self.btn_extract, self.btn_export]),
            HBox([self.btn_prev, self.btn_next]),
            HBox([self.message]), HBox([self.status])
        ])
        content = HBox([self.image_out, self.table_out], layout=Layout(width='100%'))
        display(VBox([controls, content]))

    def _find_groups_and_exams(self):
        self.status.value = 'Buscando grupos y exámenes...'
        groups = [d.name for d in self.base_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]
        self.dropdown_group.options = sorted(groups)

    def _on_group_change(self, change):
        grp = change['new']
        if not grp: return
        prac_dir = self.base_dir / grp
        self.dropdown_practice.options = sorted([d.name for d in prac_dir.iterdir() if d.is_dir() and d.name.startswith('Practica_')])
        self._update_student_dropdowns()

    def _on_practice_change(self, change):
        prac = change['new']
        if not prac: return
        ex_dir = self.base_dir / self.dropdown_group.value / prac
        self.dropdown_exam.options = sorted([p.name for p in ex_dir.glob('*.pdf')])

    def _on_exam_change(self, change):
        ex = change['new']
        if not ex: return
        # show PDF, extract grade, load last
        self.current_pdf = self.base_dir / self.dropdown_group.value / self.dropdown_practice.value / ex
        self._show_exam_image()
        self._extract_grade()
        self._save_last()
        # Auto-select student AFTER loading the new exam
        self._auto_select_student()

    def _update_student_dropdowns(self):
        # filter df by group
        if 'Grupos' in self.df_students.columns:
            grp = self.dropdown_group.value
            df_grp = self.df_students[self.df_students['Grupos'] == grp]
        else:
            df_grp = self.df_students
        last_opts = sorted(df_grp['Apellido(s)'].dropna().unique())
        self.dropdown_last.options = last_opts
        if last_opts:
            self.dropdown_last.value = last_opts[0]
            self._update_first_options()
        else:
            self.dropdown_last.value = None
            self.dropdown_first.options = []

    def _update_first_options(self):
        last = self.dropdown_last.value
        if last and not self.df_students.empty:
            names = sorted(self.df_students[self.df_students['Apellido(s)'] == last]['Nombre'].unique())
            self.dropdown_first.options = names
            if names:
                self.dropdown_first.value = names[0]
        else:
            self.dropdown_first.options = []
            
    def _auto_select_student(self):
        """Selecciona automáticamente el estudiante basado en el nombre del archivo"""
        # Obtener nombre del archivo actual
        filename = self.dropdown_exam.value
        if not filename:
            return
        
        # Función para normalizar texto (quitar acentos y mayúsculas)
        def normalize(text):
            import unicodedata
            if not text: return ""
            text = unicodedata.normalize('NFD', text.upper())
            return ''.join(c for c in text if not unicodedata.combining(c))
        
        # Extraer primera palabra (apellido)
        first_word = filename.split('_')[0] if '_' in filename else filename.split('.')[0]
        first_word_norm = normalize(first_word)
        
        # 1. PRIORIDAD ABSOLUTA: Buscar apellido que comience exactamente con la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            
            # Normalizar el apellido del dropdown
            last_norm = normalize(last)
            last_parts = last_norm.split()
            
            # Si el apellido comienza exactamente con la primera palabra del archivo
            if last_parts and last_parts[0] == first_word_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 2. SEGUNDA PRIORIDAD: Buscar apellido que contenga la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            last_norm = normalize(last)
            
            # Si el apellido contiene la primera palabra completa
            if first_word_norm in last_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 3. ÚLTIMA OPCIÓN: Búsqueda por coincidencia parcial si las anteriores fallan
        # Buscar por tokens en todo el nombre del archivo
        filename_parts = filename.replace('.pdf', '').split('_')
        
        for part in filename_parts:
            part_norm = normalize(part)
            if len(part_norm) < 3:  # Ignorar partes muy cortas
                continue
            
            for last in self.dropdown_last.options:
                if not last: continue
                last_norm = normalize(last)
                
                if part_norm in last_norm:
                    self.dropdown_last.value = last
                    self._update_first_options()
                    return

    def _navigate(self, step):
        opts = list(self.dropdown_exam.options)
        if not opts: return
        idx = opts.index(self.dropdown_exam.value) + step
        if 0 <= idx < len(opts):
            self.dropdown_exam.value = opts[idx]
        else:
            self.message.value = '<span style="color:orange;">No hay más exámenes</span>'

    def _show_exam_image(self):
        self.image_out.clear_output()
        with self.image_out:
            imgs = convert_from_path(self.current_pdf, first_page=1, last_page=2, dpi=150, fmt='jpeg')
            if not imgs:
                print('No hay imágenes')
            else:
                cropped = [im.crop((0, 0, im.width, 225)) for im in imgs]
                fig, axes = plt.subplots(nrows=len(cropped), figsize=(8, 2 * len(cropped)), tight_layout=True)
                if len(cropped) == 1:
                    axes = [axes]
                for ax, im in zip(axes, cropped):
                    ax.imshow(im)
                    ax.axis('off')
                plt.show()
        self._update_student_table()

    def _extract_grade(self):
        if not hasattr(self, 'current_pdf') or not self.current_pdf:
            return
        self.status.value = 'Extrayendo nota...'
        for p in (1, 2):
            imgs = convert_from_path(self.current_pdf, first_page=p, last_page=p, dpi=150, fmt='jpeg')
            if not imgs:
                continue
            buf = io.BytesIO()
            imgs[0].save(buf, 'JPEG')
            img_b64 = base64.b64encode(buf.getvalue()).decode()
            payload = {
                'model': self.model,
                'response_format': {'type': 'json_object'},
                'messages': [
                    {'role': 'system', 'content': 'Extract grade.'},
                    {'role': 'user', 'content': [
                        {'type': 'text', 'text': self.prompt},
                        {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{img_b64}'}}
                    ]}
                ],
                'max_tokens': 150
            }
            r = requests.post('https://api.openai.com/v1/chat/completions', headers=self.headers, json=payload)
            if r.status_code != 200:
                continue
            data = json.loads(r.json()['choices'][0]['message']['content'])
            try:
                grd = float(str(data.get('grade')).replace(',', '.'))
            except:
                continue
            self.current_grade = grd; self.current_grade_type = 'auto'
            self.grade_value.value = f"<span style='color:#000088;font-weight:bold;'>{grd:.2f}</span>"
            self.grade_input.value = grd
            self.status.value = f'Nota extraída pág {p}'
            break

    def _on_save_grade(self, b):
        g = self.grade_input.value
        if g < 0 or g > 10:
            self.message.value = '<span style="color:red;">Nota 0-10</span>'
            return
        # Update grade widgets
        self.current_grade = g; self.current_grade_type = 'manual'
        self.grade_value.value = f"<span style='color:#008800;font-weight:bold;'>{g:.2f}</span>"

        # Determine column
        m = re.search(r"Practica[_ ]?(\d+)", self.dropdown_practice.value or "")
        col = f"Examen Practica {m.group(1)}" if m else None
        last = self.dropdown_last.value; first = self.dropdown_first.value
        if col and col in self.df_students.columns:
            mask = (self.df_students['Apellido(s)'] == last) & (self.df_students['Nombre'] == first)
            if mask.any():
                self.df_students.loc[mask, col] = g
        self.message.value = f'<span style="color:green;">Nota {g:.2f} guardada</span>'
        self._update_student_table()

    def _update_student_table(self):
        if self.df_students.empty:
            return
        self.table_out.clear_output()
        with self.table_out:
            df = self.df_students
            cols = ['Nombre', 'Apellido(s)'] + [c for c in df.columns if c.startswith('Examen Practica')] + (['grade'] if 'grade' in df.columns else [])
            last = self.dropdown_last.value; first = self.dropdown_first.value
            mask = (df['Apellido(s)'] == last) & (df['Nombre'] == first)
            if mask.any():
                idx_list = list(df.index)
                pos = idx_list.index(df[mask].index[0])
                start = max(0, pos - 2)
                end = min(len(idx_list), pos + 3)
                display(df.loc[idx_list[start:end], cols])
            else:
                display(df[cols].head(5))

    def _on_export(self, b):
        out_file = self.last_dir / 'students.xlsx'
        self.df_students.to_excel(out_file, index=False)
        self.message.value = f'<span style="color:green;">Exportado a {out_file}</span>'

    def _save_last(self):
        state = {
            'group': self.dropdown_group.value,
            'practice': self.dropdown_practice.value,
            'exam': self.dropdown_exam.value,
            'last': self.dropdown_last.value,
            'first': self.dropdown_first.value
        }
        self.last_state_path.write_text(json.dumps(state))

    def _load_last(self):
        if not self.last_state_path.exists():
            return
        try:
            data = json.loads(self.last_state_path.read_text())
            if data.get('group') in self.dropdown_group.options:
                self.dropdown_group.value = data['group']
            if data.get('practice') in self.dropdown_practice.options:
                self.dropdown_practice.value = data['practice']
            if data.get('exam') in self.dropdown_exam.options:
                self.dropdown_exam.value = data['exam']
            if data.get('last') in self.dropdown_last.options:
                self.dropdown_last.value = data['last']
                self._update_first_options()
            if data.get('first') in self.dropdown_first.options:
                self.dropdown_first.value = data['first']
        except:
            pass

In [83]:
prompt = """
    Extract the grade of this exam: 
    - It is a number between 0 and 10 
    - at most two decimals, may be written with ',' or '.'), 
    - It is written with red at the top of the image
    - It is normally preceded by 'Nota final'
    If you see multiple numbers in the image:
    - take the biggest one as the final grade, at one corner of the paper   
    The field of your JSON output will have the name "grade".
"""

reviewer = ExamGradeReviewer(
    base_dir="../data/examenes_corregidos/",
    model   ="gpt-4o",
    #model="gpt-3.5-turbo",
    df_students=df_students_info_full,
    prompt=prompt
)

In [207]:
df_students_info_full

,Nombre,Apellido(s),Dirección de correo,Grupos,Presentada_3,Comentario_3,Presentada_5,Comentario_5,Examen Practica 3,Examen Practica 5,Examen Practica 3 entregado,Examen Practica 5 entregado
0,SOFIA,AGAPITO DELGADO,s.agapito@alumnos.upm.es,IWSIT11,0,NP. Examen no presentado,1,Examen no presentado,None,None,0,0
1,LLOYD DAREN,AGUILAR DESIAR,daren.aguilar@alumnos.upm.es,IWSIM12,0,NP. Examen no presentado,1,Examen no presentado,None,None,0,0
2,JAVIER,AGUIRRE HERVIAS,javier.aguirrehervias@alumnos.upm.es,IWSIM12,0,NP. Examen no presentado,1,,None,None,0,1
3,MATEO,ALBRIZIO,mateo.albrizio@alumnos.upm.es,IWSIM12,0,NP. Examen no presentado,1,Examen no presentado,None,None,0,0
4,NICOLAS,ALONSO FERNANDEZ,ni.alonso@alumnos.upm.es,IWSIM12,0,NP. Examen no presentado,0,NP. Examen no presentado,None,None,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
439,HAOQING,ZHANG,haoqing.zhang@alumnos.upm.es,CITIT11,0,NP. Examen no presentado,0,NP. Examen no presentado,None,None,0,0
440,JIONGHAO,ZHANG,jionghao.zhang@alumnos.upm.es,CITIM12,1,Examen no presentado,1,,None,None,0,1
441,STEVEN WEI,ZHANG XIA,steven.zhang@alumnos.upm.es,CITIT11,0,NP. Examen no presentado,1,Examen no presentado,None,None,0,0
442,YI,ZHOU,yi.zhou@alumnos.upm.es,CITIT11,0,NP. Examen no presentado,1,Examen no presentado,None,None,0,0


In [208]:
import os
import io
import json
import base64
import re
import requests
from pathlib import Path
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown, FloatText, HTML, IntProgress
from IPython.display import display
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor

class BatchExamGradeProcessor:
    def __init__(
        self,
        base_dir: str = "../data/examenes_corregidos/",
        model: str = "gpt-4o",
        df_students: pd.DataFrame = None,
        prompt: str = None,
        max_workers: int = 4
    ):
        # Directorios y estado inicial
        self.base_dir = Path(base_dir)
        self.last_dir = Path("./data/last"); self.last_dir.mkdir(exist_ok=True, parents=True)
        self.last_state_path = self.last_dir / "last_visit.json"
        self.processed_grades_path = self.last_dir / "processed_grades.json"

        # Config OpenAI
        self.model = model
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY no está configurada")
        self.prompt = prompt or (
            "Extract the grade of this exam:\n"
            "- It is a number between 0 and 10\n"
            "- At most two decimals (',' or '.')\n"
            "- Written in red at the top, normally preceded by 'Nota final'\n"
            "If multiple numbers, take the largest at a corner.\n"
            "Output JSON with field 'grade'."
        )
        self.headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.api_key}"}

        # Data students
        self.df_students = df_students.copy() if df_students is not None else pd.DataFrame()
        
        # Procesamiento por lotes
        self.max_workers = max_workers
        self.processed_exams = {}
        self.load_processed_exams()

        # UI para procesamiento
        self.output = Output()
        self.status_label = Label()
        self.progress = IntProgress(min=0, max=100, description='Progreso:')
        self.btn_process = Button(description='Procesar exámenes', button_style='primary')
        self.btn_process.on_click(self._on_process_click)
        
        # Revisor interactivo
        self.reviewer = None
        self.btn_review = Button(description='Revisar resultados', button_style='success')
        self.btn_review.on_click(self._on_review_click)
        
        # Mostrar interfaz inicial
        self._show_batch_interface()
    
    def _show_batch_interface(self):
        """Muestra la interfaz para el procesamiento por lotes"""
        display(VBox([
            Label(value="Procesamiento por lotes de exámenes"),
            HBox([self.btn_process, self.btn_review]),
            self.status_label,
            self.progress,
            self.output
        ]))
        
        # Mostrar estadísticas de procesamiento si hay datos
        if self.processed_exams:
            self._show_processing_stats()
    
    def _show_processing_stats(self):
        """Muestra estadísticas sobre exámenes procesados"""
        with self.output:
            total = len(self.processed_exams)
            grades_found = sum(1 for info in self.processed_exams.values() if info.get('grade') is not None)
            
            print(f"Exámenes procesados: {total}")
            print(f"Notas extraídas: {grades_found} ({grades_found/total*100:.1f}% de éxito)")
            
            # Estadísticas por grupo
            if total > 0:
                groups = {}
                for path, info in self.processed_exams.items():
                    group = info.get('group', 'Desconocido')
                    practice = info.get('practice', 'Desconocida')
                    key = f"{group}/{practice}"
                    if key not in groups:
                        groups[key] = {'total': 0, 'with_grade': 0}
                    groups[key]['total'] += 1
                    if info.get('grade') is not None:
                        groups[key]['with_grade'] += 1
                
                print("\nEstadísticas por grupo y práctica:")
                for key, stats in groups.items():
                    success_rate = stats['with_grade'] / stats['total'] * 100 if stats['total'] > 0 else 0
                    print(f"{key}: {stats['with_grade']}/{stats['total']} ({success_rate:.1f}%)")
    
    def _on_process_click(self, b):
        """Inicia el procesamiento por lotes de exámenes"""
        self.output.clear_output()
        self.status_label.value = "Buscando exámenes..."
        
        with self.output:
            # Encontrar todos los exámenes
            all_exams = self._find_all_exams()
            
            if not all_exams:
                print("No se encontraron exámenes para procesar.")
                return
            
            # Filtrar exámenes ya procesados si el usuario lo desea
            if self.processed_exams:
                reprocess = input("¿Desea reprocesar exámenes ya procesados? (s/n): ").lower() == 's'
                if not reprocess:
                    all_exams = [exam for exam in all_exams if str(exam['path']) not in self.processed_exams]
            
            total_exams = len(all_exams)
            print(f"Se procesarán {total_exams} exámenes.")
            
            if total_exams == 0:
                self.status_label.value = "Todos los exámenes ya han sido procesados"
                return
                
            self.progress.max = total_exams
            self.progress.value = 0
            
            # Procesar exámenes
            start_time = time.time()
            self._process_exams_batch(all_exams)
            elapsed_time = time.time() - start_time
            
            print(f"\nProcesamiento completado en {elapsed_time:.1f} segundos.")
            print(f"Velocidad promedio: {total_exams / elapsed_time:.2f} exámenes por segundo.")
            
            # Guardar resultados
            self.save_processed_exams()
            self._show_processing_stats()
            
            # Actualizar DataFrame si está disponible
            if not self.df_students.empty:
                self._update_dataframe_with_grades()
                print("\nDataFrame actualizado con las notas extraídas.")
            
            self.status_label.value = "Procesamiento completado"
    
    def _find_all_exams(self):
        """Encuentra todos los exámenes en la estructura de directorios"""
        all_exams = []
        
        for group_dir in self.base_dir.iterdir():
            if not group_dir.is_dir():
                continue
                
            for practice_dir in group_dir.iterdir():
                if not practice_dir.is_dir() or not practice_dir.name.startswith("Practica_"):
                    continue
                    
                practice_num = practice_dir.name.replace("Practica_", "")
                
                for pdf_file in practice_dir.glob("*.pdf"):
                    all_exams.append({
                        'path': pdf_file,
                        'group': group_dir.name,
                        'practice': practice_dir.name,
                        'practice_num': practice_num,
                        'filename': pdf_file.name
                    })
        
        return all_exams
    
    def _process_exams_batch(self, exams):
        """Procesa los exámenes en paralelo utilizando ThreadPoolExecutor"""
        self.status_label.value = "Procesando exámenes..."
        
        processed_count = 0
        
        # Función para procesar un examen individual y actualizar el progreso
        def process_and_update(exam):
            result = self._extract_grade_from_pdf(exam['path'])
            
            # Guardar resultado con metadatos
            exam_info = {
                'group': exam['group'],
                'practice': exam['practice'],
                'practice_num': exam['practice_num'],
                'filename': exam['filename'],
                'grade': result['grade'] if result else None,
                'confidence': result.get('confidence', 0) if result else 0,
                'processed_time': time.time()
            }
            
            self.processed_exams[str(exam['path'])] = exam_info
            
            # Actualizar progreso
            nonlocal processed_count
            processed_count += 1
            self.progress.value = processed_count
            
            return exam_info
        
        # Procesar en paralelo
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            results = list(executor.map(process_and_update, exams))
        
        return results
    
    def _extract_grade_from_pdf(self, pdf_path):
        """Extrae la nota de un PDF utilizando la API de OpenAI"""
        try:
            for page in (1, 2):  # Intentar con las primeras dos páginas
                try:
                    # Convertir la página a imagen
                    imgs = convert_from_path(pdf_path, first_page=page, last_page=page, dpi=150, fmt='jpeg')
                    if not imgs:
                        continue
                        
                    # Preparar la imagen para la API
                    buf = io.BytesIO()
                    imgs[0].save(buf, 'JPEG')
                    img_b64 = base64.b64encode(buf.getvalue()).decode()
                    
                    # Preparar la solicitud a la API
                    payload = {
                        'model': self.model,
                        'response_format': {'type': 'json_object'},
                        'messages': [
                            {'role': 'system', 'content': 'Extract grade.'},
                            {'role': 'user', 'content': [
                                {'type': 'text', 'text': self.prompt},
                                {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{img_b64}'}}
                            ]}
                        ],
                        'max_tokens': 150
                    }
                    
                    # Realizar la solicitud
                    r = requests.post('https://api.openai.com/v1/chat/completions', 
                                     headers=self.headers, json=payload)
                    
                    if r.status_code != 200:
                        continue
                    
                    data = json.loads(r.json()['choices'][0]['message']['content'])
                    
                    # Extraer y validar la nota
                    if 'grade' in data:
                        try:
                            grade = float(str(data['grade']).replace(',', '.'))
                            if 0 <= grade <= 10:
                                return {
                                    'grade': grade,
                                    'page': page,
                                    'confidence': data.get('confidence', 1.0)
                                }
                        except (ValueError, TypeError):
                            pass
                except Exception as e:
                    print(f"Error procesando {pdf_path.name}, página {page}: {str(e)}")
                    continue
            
            return None
        except Exception as e:
            print(f"Error general procesando {pdf_path.name}: {str(e)}")
            return None
    
    def _update_dataframe_with_grades(self):
        """Actualiza el DataFrame con las notas extraídas"""
        if self.df_students.empty:
            return
        
        changes = 0
        
        for _, exam_info in self.processed_exams.items():
            if exam_info.get('grade') is None:
                continue
            
            # Obtener información del examen
            group = exam_info['group']
            practice_num = exam_info.get('practice_num')
            filename = exam_info['filename']
            grade = exam_info['grade']
            
            if not practice_num:
                continue
                
            # Determinar columna
            column = f"Examen Practica {practice_num}"
            
            if column not in self.df_students.columns:
                continue
            
            # Extraer apellido y nombre del nombre del archivo
            parts = filename.replace('.pdf', '').split('_')
            if len(parts) < 2:
                continue
                
            apellido = parts[0]
            nombre = parts[1] if len(parts) > 1 else ""
            
            # Normalizar para la búsqueda
            def normalize(text):
                import unicodedata
                text = unicodedata.normalize('NFD', text.upper())
                return ''.join(c for c in text if not unicodedata.combining(c))
            
            apellido_norm = normalize(apellido)
            nombre_norm = normalize(nombre)
            
            # Filtrar por grupo
            df_group = self.df_students[self.df_students['Grupos'] == group]
            
            # Buscar coincidencia
            for idx, row in df_group.iterrows():
                last_norm = normalize(row['Apellido(s)'])
                first_norm = normalize(row['Nombre'])
                
                if apellido_norm in last_norm or last_norm in apellido_norm:
                    if not nombre or nombre_norm in first_norm or first_norm in nombre_norm:
                        # Verificar si la nota ya estaba
                        current_grade = self.df_students.loc[idx, column]
                        if pd.isna(current_grade) or current_grade == 0:
                            self.df_students.loc[idx, column] = grade
                            changes += 1
                            break
        
        print(f"Se actualizaron {changes} notas en el DataFrame.")
    
    def load_processed_exams(self):
        """Carga exámenes procesados de un archivo JSON"""
        if self.processed_grades_path.exists():
            try:
                self.processed_exams = json.loads(self.processed_grades_path.read_text())
                return True
            except Exception as e:
                print(f"Error cargando exámenes procesados: {e}")
        return False
    
    def save_processed_exams(self):
        """Guarda exámenes procesados en un archivo JSON"""
        try:
            self.processed_grades_path.write_text(json.dumps(self.processed_exams, indent=2))
            return True
        except Exception as e:
            print(f"Error guardando exámenes procesados: {e}")
            return False
    
    def _on_review_click(self, b):
        """Muestra el revisor interactivo"""
        if self.reviewer is None:
            self.reviewer = ExamGradeReviewer(
                base_dir=self.base_dir,
                model=self.model,
                df_students=self.df_students,
                processed_exams=self.processed_exams,
                save_callback=self.save_processed_exams
            )
        else:
            with self.output:
                print("Mostrando revisor interactivo...")
                display(self.reviewer.get_interface())

class ExamGradeReviewer:
    def __init__(
        self,
        base_dir: str = "../data/examenes_corregidos/",
        model: str = "gpt-4o",
        df_students: pd.DataFrame = None,
        processed_exams: dict = None,
        save_callback = None
    ):
        # Directorios y estado inicial
        self.base_dir = Path(base_dir)
        self.last_dir = Path("./data/last"); self.last_dir.mkdir(exist_ok=True, parents=True)
        self.last_state_path = self.last_dir / "last_visit.json"

        # Config OpenAI
        self.model = model
        self.api_key = os.environ.get("OPENAI_API_KEY")
        
        # Data students y exámenes procesados
        self.df_students = df_students.copy() if df_students is not None else pd.DataFrame()
        self.processed_exams = processed_exams or {}
        self.save_callback = save_callback

        # Dropdown widgets
        self.dropdown_group = Dropdown(description='Grupo:', layout=Layout(width='200px'))
        self.dropdown_practice = Dropdown(description='Práctica:', layout=Layout(width='200px'))
        self.dropdown_exam = Dropdown(description='Examen:', layout=Layout(width='350px'))
        self.dropdown_last = Dropdown(description='Apellido:', layout=Layout(width='200px'))
        self.dropdown_first = Dropdown(description='Nombre:', layout=Layout(width='200px'))

        # Grade widgets
        self.grade_label = Label('Nota:', layout=Layout(width='50px'))
        self.grade_value = HTML(layout=Layout(width='80px'))
        self.grade_input = FloatText(0.0, min=0, max=10, step=0.1, layout=Layout(width='80px'))
        self.btn_save_grade = Button(description='Guardar nota', button_style='success', layout=Layout(width='120px'))
        self.btn_extract = Button(description='Re-extraer', button_style='info', layout=Layout(width='120px'))
        self.btn_export = Button(description='Exportar Excel', button_style='primary', layout=Layout(width='150px'))
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='100px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='100px'))

        # Output areas
        self.image_out = Output(layout=Layout(width='60%', overflow='hidden'))
        self.table_out = Output(layout=Layout(width='40%', overflow='auto'))
        self.status = Label()
        self.message = HTML()

        # Bind events
        self.dropdown_group.observe(self._on_group_change, names='value')
        self.dropdown_practice.observe(self._on_practice_change, names='value')
        self.dropdown_exam.observe(self._on_exam_change, names='value')
        self.dropdown_last.observe(lambda _: self._update_first_options(), names='value')
        self.btn_prev.on_click(lambda b: self._navigate(-1))
        self.btn_next.on_click(lambda b: self._navigate(1))
        self.btn_extract.on_click(lambda b: self._extract_grade())
        self.btn_save_grade.on_click(self._on_save_grade)
        self.btn_export.on_click(self._on_export)

        # Build interface and initialize
        self.interface = self._build_interface()
        self._find_groups_and_exams()
        self._load_last()
    
    def get_interface(self):
        """Devuelve la interfaz completa para mostrarla"""
        return self.interface

    def _build_interface(self):
        controls = VBox([
            HBox([self.dropdown_group, self.dropdown_practice, self.dropdown_exam]),
            HBox([self.dropdown_last, self.dropdown_first]),
            HBox([self.grade_label, self.grade_value, self.grade_input,
                  self.btn_save_grade, self.btn_extract, self.btn_export]),
            HBox([self.btn_prev, self.btn_next]),
            HBox([self.message]), HBox([self.status])
        ])
        content = HBox([self.image_out, self.table_out], layout=Layout(width='100%'))
        return VBox([controls, content])

    def _find_groups_and_exams(self):
        self.status.value = 'Buscando grupos y exámenes...'
        groups = [d.name for d in self.base_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]
        self.dropdown_group.options = sorted(groups)

    def _on_group_change(self, change):
        grp = change['new']
        if not grp: return
        prac_dir = self.base_dir / grp
        self.dropdown_practice.options = sorted([d.name for d in prac_dir.iterdir() if d.is_dir() and d.name.startswith('Practica_')])
        self._update_student_dropdowns()

    def _on_practice_change(self, change):
        prac = change['new']
        if not prac: return
        ex_dir = self.base_dir / self.dropdown_group.value / prac
        self.dropdown_exam.options = sorted([p.name for p in ex_dir.glob('*.pdf')])

    def _on_exam_change(self, change):
        ex = change['new']
        if not ex: return
        # show PDF, extract grade, load last
        self.current_pdf = self.base_dir / self.dropdown_group.value / self.dropdown_practice.value / ex
        self._show_exam_image()
        
        # Cargar nota pre-procesada en lugar de extraer en tiempo real
        self._load_preprocessed_grade()
        
        self._save_last()
        # Auto-select student AFTER loading the new exam
        self._auto_select_student()

    def _update_student_dropdowns(self):
        # filter df by group
        if 'Grupos' in self.df_students.columns:
            grp = self.dropdown_group.value
            df_grp = self.df_students[self.df_students['Grupos'] == grp]
        else:
            df_grp = self.df_students
        last_opts = sorted(df_grp['Apellido(s)'].dropna().unique())
        self.dropdown_last.options = last_opts
        if last_opts:
            self.dropdown_last.value = last_opts[0]
            self._update_first_options()
        else:
            self.dropdown_last.value = None
            self.dropdown_first.options = []

    def _update_first_options(self):
        last = self.dropdown_last.value
        if last and not self.df_students.empty:
            names = sorted(self.df_students[self.df_students['Apellido(s)'] == last]['Nombre'].unique())
            self.dropdown_first.options = names
            if names:
                self.dropdown_first.value = names[0]
        else:
            self.dropdown_first.options = []
            
    def _load_preprocessed_grade(self):
        """Carga la nota pre-procesada si existe"""
        if not hasattr(self, 'current_pdf') or not self.current_pdf:
            return
            
        path_str = str(self.current_pdf)
        if path_str in self.processed_exams:
            exam_info = self.processed_exams[path_str]
            if exam_info.get('grade') is not None:
                self.current_grade = exam_info['grade']
                self.current_grade_type = 'auto'
                self.grade_value.value = f"<span style='color:#000088;font-weight:bold;'>{exam_info['grade']:.2f}</span>"
                self.grade_input.value = exam_info['grade']
                self.status.value = f"Nota pre-procesada: {exam_info['grade']:.2f}"
                return
        
        # Si no hay nota pre-procesada
        self.grade_value.value = "<span style='color:#cc0000;'>Sin nota</span>"
        self.grade_input.value = 0.0
        self.status.value = "No hay nota pre-procesada. Use 'Re-extraer' para intentar OCR."
            
    def _auto_select_student(self):
        """Selecciona automáticamente el estudiante basado en el nombre del archivo"""
        # Obtener nombre del archivo actual
        filename = self.dropdown_exam.value
        if not filename:
            return
        
        # Función para normalizar texto (quitar acentos y mayúsculas)
        def normalize(text):
            import unicodedata
            if not text: return ""
            text = unicodedata.normalize('NFD', text.upper())
            return ''.join(c for c in text if not unicodedata.combining(c))
        
        # Extraer primera palabra (apellido)
        first_word = filename.split('_')[0] if '_' in filename else filename.split('.')[0]
        first_word_norm = normalize(first_word)
        
        # 1. PRIORIDAD ABSOLUTA: Buscar apellido que comience exactamente con la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            
            # Normalizar el apellido del dropdown
            last_norm = normalize(last)
            last_parts = last_norm.split()
            
            # Si el apellido comienza exactamente con la primera palabra del archivo
            if last_parts and last_parts[0] == first_word_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 2. SEGUNDA PRIORIDAD: Buscar apellido que contenga la primera palabra
        for last in self.dropdown_last.options:
            if not last: continue
            last_norm = normalize(last)
            
            # Si el apellido contiene la primera palabra completa
            if first_word_norm in last_norm:
                self.dropdown_last.value = last
                self._update_first_options()
                return
        
        # 3. ÚLTIMA OPCIÓN: Búsqueda por coincidencia parcial si las anteriores fallan
        # Buscar por tokens en todo el nombre del archivo
        filename_parts = filename.replace('.pdf', '').split('_')
        
        for part in filename_parts:
            part_norm = normalize(part)
            if len(part_norm) < 3:  # Ignorar partes muy cortas
                continue
            
            for last in self.dropdown_last.options:
                if not last: continue
                last_norm = normalize(last)
                
                if part_norm in last_norm:
                    self.dropdown_last.value = last
                    self._update_first_options()
                    return

    def _navigate(self, step):
        opts = list(self.dropdown_exam.options)
        if not opts: return
        idx = opts.index(self.dropdown_exam.value) + step
        if 0 <= idx < len(opts):
            self.dropdown_exam.value = opts[idx]
        else:
            self.message.value = '<span style="color:orange;">No hay más exámenes</span>'

    def _show_exam_image(self):
        self.image_out.clear_output()
        with self.image_out:
            imgs = convert_from_path(self.current_pdf, first_page=1, last_page=2, dpi=150, fmt='jpeg')
            if not imgs:
                print('No hay imágenes')
            else:
                cropped = [im.crop((0, 0, im.width, 225)) for im in imgs]
                fig, axes = plt.subplots(nrows=len(cropped), figsize=(8, 2 * len(cropped)), tight_layout=True)
                if len(cropped) == 1:
                    axes = [axes]
                for ax, im in zip(axes, cropped):
                    ax.imshow(im)
                    ax.axis('off')
                plt.show()
        self._update_student_table()

    def _extract_grade(self):
        """Extrae la nota usando OCR solo cuando se solicita explícitamente"""
        if not hasattr(self, 'current_pdf') or not self.current_pdf:
            return
        
        self.status.value = 'Re-extrayendo nota...'
        
        # Aquí reutilizamos la lógica de extracción del procesador por lotes
        result = BatchExamGradeProcessor(model=self.model)._extract_grade_from_pdf(self.current_pdf)
        
        if result and result.get('grade') is not None:
            grade = result['grade']
            # Actualizar la interfaz
            self.current_grade = grade
            self.current_grade_type = 'auto'
            self.grade_value.value = f"<span style='color:#000088;font-weight:bold;'>{grade:.2f}</span>"
            self.grade_input.value = grade
            self.status.value = f"Nota extraída: {grade:.2f}"
            
            # Actualizar el registro de exámenes procesados
            path_str = str(self.current_pdf)
            if path_str in self.processed_exams:
                self.processed_exams[path_str]['grade'] = grade
            else:
                # Obtener metadatos
                group = self.dropdown_group.value
                practice = self.dropdown_practice.value
                practice_num = practice.replace("Practica_", "") if practice else ""
                
                self.processed_exams[path_str] = {
                    'group': group,
                    'practice': practice,
                    'practice_num': practice_num,
                    'filename': self.current_pdf.name,
                    'grade': grade,
                    'processed_time': time.time()
                }
            
            # Guardar cambios
            if self.save_callback:
                self.save_callback()
        else:
            self.status.value = "No se pudo extraer la nota"

    def _on_save_grade(self, b):
        g = self.grade_input.value
        if g < 0 or g > 10:
            self.message.value = '<span style="color:red;">Nota 0-10</span>'
            return
        # Update grade widgets
        self.current_grade = g
        self.current_grade_type = 'manual'
        self.grade_value.value = f"<span style='color:#008800;font-weight:bold;'>{g:.2f}</span>"

        # Determine column
        m = re.search(r"Practica[_ ]?(\d+)", self.dropdown_practice.value or "")
        col = f"Examen Practica {m.group(1)}" if m else None
        last = self.dropdown_last.value
        first = self.dropdown_first.value
        
        # Actualizar el dataframe
        if col and col in self.df_students.columns:
            mask = (self.df_students['Apellido(s)'] == last) & (self.df_students['Nombre'] == first)
            if mask.any():
                self.df_students.loc[mask, col] = g
        
        # Actualizar examen procesado
        if hasattr(self, 'current_pdf') and self.current_pdf:
            path_str = str(self.current_pdf)
            if path_str in self.processed_exams:
                self.processed_exams[path_str]['grade'] = g
                self.processed_exams[path_str]['manual'] = True
            
            # Guardar cambios
            if self.save_callback:
                self.save_callback()
        
        self.message.value = f'<span style="color:green;">Nota {g:.2f} guardada</span>'
        self._update_student_table()

    def _update_student_table(self):
        if self.df_students.empty:
            return
        self.table_out.clear_output()
        with self.table_out:
            df = self.df_students
            cols = ['Nombre', 'Apellido(s)'] + [c for c in df.columns if c.startswith('Examen Practica')] + (['grade'] if 'grade' in df.columns else [])
            last = self.dropdown_last.value
            first = self.dropdown_first.value
            mask = (df['Apellido(s)'] == last) & (df['Nombre'] == first)
            if mask.any():
                idx_list = list(df.index)
                pos = idx_list.index(df[mask].index[0])
                start = max(0, pos - 2)
                end = min(len(idx_list), pos + 3)
                display(df.loc[idx_list[start:end], cols])
            else:
                display(df[cols].head(5))

    def _on_export(self, b):
        out_file = self.last_dir / 'students.xlsx'
        self.df_students.to_excel(out_file, index=False)
        self.message.value = f'<span style="color:green;">Exportado a {out_file}</span>'

    def _save_last(self):
        state = {
            'group': self.dropdown_group.value,
            'practice': self.dropdown_practice.value,
            'exam': self.dropdown_exam.value,
            'last': self.dropdown_last.value,
            'first': self.dropdown_first.value
        }
        self.last_state_path.write_text(json.dumps(state))

    def _load_last(self):
        if not self.last_state_path.exists():
            return
        try:
            data = json.loads(self.last_state_path.read_text())
            if data.get('group') in self.dropdown_group.options:
                self.dropdown_group.value = data['group']
            if data.get('practice') in self.dropdown_practice.options:
                self.dropdown_practice.value = data['practice']
            if data.get('exam') in self.dropdown_exam.options:
                self.dropdown_exam.value = data['exam']
            if data.get('last') in self.dropdown_last.options:
                self.dropdown_last.value = data['last']
                self._update_first_options()
            if data.get('first') in self.dropdown_first.options:
                self.dropdown_first.value = data['first']
        except:
            pass

In [216]:
reviewer = BatchExamGradeProcessor(
    base_dir="../data/examenes_corregidos/",
    model="gpt-4o",
    df_students=df_students_info_full,
)


In [217]:
import os
import io
import json
import base64
import re
import requests
from pathlib import Path
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout, Label, Dropdown, FloatText, HTML
from IPython.display import display
import pandas as pd

class SimpleExamGradeReviewer:
    def __init__(
        self,
        base_dir: str = "../data/examenes_corregidos/",
        model: str = "gpt-4o",
        df_students: pd.DataFrame = None,
        prompt: str = None
    ):
        # Directorios y estado inicial
        self.base_dir = Path(base_dir)
        self.last_dir = Path("./data/last")
        self.last_dir.mkdir(exist_ok=True, parents=True)
        self.last_state_path = self.last_dir / "last_visit.json"

        # Config OpenAI
        self.model = model
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY no está configurada")
        self.prompt = prompt or (
            "Extract the grade of this exam:\n"
            "- It is a number between 0 and 10\n"
            "- At most two decimals (',' or '.')\n"
            "- Written in red at the top, normally preceded by 'Nota final'\n"
            "Output JSON with field 'grade'."
        )
        self.headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.api_key}"}

        # Data students
        self.df_students = df_students.copy() if df_students is not None else pd.DataFrame()
        
        # Estado interno
        self.current_pdf = None
        self.current_grade = None
        self.current_grade_type = 'auto'

        # Widgets
        self.setup_ui()
        self._find_groups_and_exams()
        
    def setup_ui(self):
        # Dropdowns
        self.dropdown_group = Dropdown(description='Grupo:', layout=Layout(width='200px'))
        self.dropdown_practice = Dropdown(description='Práctica:', layout=Layout(width='200px'))
        self.dropdown_exam = Dropdown(description='Examen:', layout=Layout(width='350px'))
        self.dropdown_last = Dropdown(description='Apellido:', layout=Layout(width='200px'))
        self.dropdown_first = Dropdown(description='Nombre:', layout=Layout(width='200px'))

        # Grade widgets
        self.grade_label = Label('Nota:', layout=Layout(width='50px'))
        self.grade_value = HTML(layout=Layout(width='80px'))
        self.grade_input = FloatText(0.0, min=0, max=10, step=0.1, layout=Layout(width='80px'))
        self.btn_save_grade = Button(description='Guardar nota', button_style='success', layout=Layout(width='120px'))
        self.btn_extract = Button(description='Extraer nota', button_style='info', layout=Layout(width='120px'))
        self.btn_export = Button(description='Exportar Excel', button_style='primary', layout=Layout(width='150px'))
        self.btn_prev = Button(description='← Anterior', layout=Layout(width='100px'))
        self.btn_next = Button(description='Siguiente →', layout=Layout(width='100px'))

        # Output areas
        self.image_out = Output(layout=Layout(width='60%', overflow='hidden'))
        self.table_out = Output(layout=Layout(width='40%', overflow='auto'))
        self.status = Label()
        self.message = HTML()
        
        # Bind events
        self.dropdown_group.observe(self._on_group_change, names='value')
        self.dropdown_practice.observe(self._on_practice_change, names='value')
        self.dropdown_exam.observe(self._on_exam_change, names='value')
        self.btn_prev.on_click(lambda b: self._navigate(-1))
        self.btn_next.on_click(lambda b: self._navigate(1))
        self.btn_extract.on_click(lambda b: self._extract_grade())
        self.btn_save_grade.on_click(self._on_save_grade)
        self.btn_export.on_click(self._on_export)
        
        # Layout principal
        controls = VBox([
            HBox([self.dropdown_group, self.dropdown_practice, self.dropdown_exam]),
            HBox([self.dropdown_last, self.dropdown_first]),
            HBox([self.grade_label, self.grade_value, self.grade_input,
                  self.btn_save_grade, self.btn_extract, self.btn_export]),
            HBox([self.btn_prev, self.btn_next]),
            HBox([self.message]), HBox([self.status])
        ])
        content = HBox([self.image_out, self.table_out], layout=Layout(width='100%'))
        self.interface = VBox([controls, content])
        
        # Mostrar la interfaz automáticamente
        display(self.interface)

    def _find_groups_and_exams(self):
        self.status.value = 'Buscando grupos y exámenes...'
        groups = [d.name for d in self.base_dir.iterdir() if d.is_dir() and not d.name.startswith('.')]
        self.dropdown_group.options = sorted(groups)

    def _on_group_change(self, change):
        grp = change['new']
        if not grp: return
        prac_dir = self.base_dir / grp
        self.dropdown_practice.options = sorted([d.name for d in prac_dir.iterdir() 
                                              if d.is_dir() and d.name.startswith('Practica_')])

    def _on_practice_change(self, change):
        prac = change['new']
        if not prac: return
        ex_dir = self.base_dir / self.dropdown_group.value / prac
        self.dropdown_exam.options = sorted([p.name for p in ex_dir.glob('*.pdf')])

    def _on_exam_change(self, change):
        ex = change['new']
        if not ex: return
        self.current_pdf = self.base_dir / self.dropdown_group.value / self.dropdown_practice.value / ex
        self._show_exam_image()
        self._extract_grade()

    def _navigate(self, step):
        opts = list(self.dropdown_exam.options)
        if not opts: return
        idx = opts.index(self.dropdown_exam.value) + step
        if 0 <= idx < len(opts):
            self.dropdown_exam.value = opts[idx]
        else:
            self.message.value = '<span style="color:orange;">No hay más exámenes</span>'

    def _show_exam_image(self):
        self.image_out.clear_output()
        with self.image_out:
            imgs = convert_from_path(self.current_pdf, first_page=1, last_page=2, dpi=150, fmt='jpeg')
            if not imgs:
                print('No hay imágenes')
            else:
                cropped = [im.crop((0, 0, im.width, 225)) for im in imgs]
                fig, axes = plt.subplots(nrows=len(cropped), figsize=(8, 2*len(cropped)), tight_layout=True)
                if len(cropped) == 1:
                    axes = [axes]
                for ax, im in zip(axes, cropped):
                    ax.imshow(im)
                    ax.axis('off')
                plt.show()

    def _extract_grade(self):
        if not hasattr(self, 'current_pdf') or not self.current_pdf:
            return
        self.status.value = 'Extrayendo nota...'
        
        # Extraer primero de la página 1, luego de la página 2
        for p in (1, 2):
            print(f"Intentando extraer nota de la página {p}...")
            # Convertir página a imagen
            imgs = convert_from_path(self.current_pdf, first_page=p, last_page=p, dpi=150, fmt='jpeg')
            if not imgs:
                continue
                
            # Convertir imagen a base64
            buf = io.BytesIO()
            imgs[0].save(buf, 'JPEG')
            img_b64 = base64.b64encode(buf.getvalue()).decode()
            
            # Preparar solicitud API
            payload = {
                'model': self.model,
                'response_format': {'type': 'json_object'},
                'messages': [
                    {'role': 'system', 'content': 'Extract grade.'},
                    {'role': 'user', 'content': [
                        {'type': 'text', 'text': self.prompt},
                        {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{img_b64}'}}
                    ]}
                ],
                'max_tokens': 150
            }
            
            # Llamar API
            print("Llamando a la API de OpenAI...")
            r = requests.post('https://api.openai.com/v1/chat/completions', headers=self.headers, json=payload)
            
            if r.status_code != 200:
                print(f"Error en API: {r.status_code} - {r.text}")
                continue
                
            # Procesar resultado
            print("Procesando respuesta...")
            result_json = r.json()
            print(f"Respuesta: {result_json}")
            
            data = json.loads(result_json['choices'][0]['message']['content'])
            print(f"Contenido extraído: {data}")
            
            # Extraer nota
            try:
                grade_str = str(data.get('grade', '')).replace(',', '.')
                print(f"Texto de nota: '{grade_str}'")
                grade = float(grade_str)
                
                # Actualizar interfaz
                self.current_grade = grade
                self.current_grade_type = 'auto'
                self.grade_value.value = f"<span style='color:#000088;font-weight:bold;'>{grade:.2f}</span>"
                self.grade_input.value = grade
                self.status.value = f'Nota extraída pág {p}: {grade:.2f}'
                
                # Actualizar estudiante
                self._update_student_table()
                return
            except Exception as e:
                print(f"Error extrayendo nota: {e}")
                continue
                
        self.status.value = 'No se pudo extraer nota'

    def _on_save_grade(self, b):
        g = self.grade_input.value
        if g < 0 or g > 10:
            self.message.value = '<span style="color:red;">Nota 0-10</span>'
            return
        
        # Update grade widgets
        self.current_grade = g
        self.current_grade_type = 'manual'
        self.grade_value.value = f"<span style='color:#008800;font-weight:bold;'>{g:.2f}</span>"
        
        # Determine column
        m = re.search(r"Practica[_ ]?(\d+)", self.dropdown_practice.value or "")
        col = f"Examen Practica {m.group(1)}" if m else None
        last = self.dropdown_last.value
        first = self.dropdown_first.value
        
        if col and col in self.df_students.columns:
            mask = (self.df_students['Apellido(s)'] == last) & (self.df_students['Nombre'] == first)
            if mask.any():
                self.df_students.loc[mask, col] = g
                
        self.message.value = f'<span style="color:green;">Nota {g:.2f} guardada</span>'
        self._update_student_table()

    def _update_student_table(self):
        if self.df_students.empty:
            return
            
        self.table_out.clear_output()
        with self.table_out:
            # Intentar extraer nombre del archivo
            if hasattr(self, 'current_pdf') and self.current_pdf:
                filename = self.current_pdf.name
                parts = filename.replace('.pdf', '').split('_')
                
                if len(parts) >= 2:
                    apellido = parts[0]
                    nombre = parts[1]
                    
                    # Buscar en DataFrame
                    df_filtro = self.df_students[
                        self.df_students['Apellido(s)'].str.contains(apellido, case=False, na=False)
                    ]
                    
                    if not df_filtro.empty:
                        # Si hay resultados, mostrar
                        cols = ['Nombre', 'Apellido(s)'] + [c for c in self.df_students.columns if c.startswith('Examen Practica')]
                        display(df_filtro[cols].head())
                    else:
                        display(pd.DataFrame([{'Mensaje': f'No se encontró alumno con apellido {apellido}'}]))
                else:
                    display(pd.DataFrame([{'Mensaje': 'Nombre de archivo no tiene formato esperado'}]))

    def _on_export(self, b):
        if self.df_students.empty:
            self.message.value = '<span style="color:red;">No hay datos para exportar</span>'
            return
            
        out_file = self.last_dir / 'students.xlsx'
        self.df_students.to_excel(out_file, index=False)
        self.message.value = f'<span style="color:green;">Exportado a {out_file}</span>'

In [218]:
# Crear el revisor con la versión simplificada
reviewer = SimpleExamGradeReviewer(
    base_dir="../data/examenes_corregidos/",
    model="gpt-4o",
    df_students=df_students_info_full,
    prompt=prompt
)
# No necesitas llamar a display(reviewer.interface) porque ya lo hace internamente